# Import librarys and Data

In [ ]:
#!pip install -r requirements.txt

In [ ]:
import sys


def check_python_version() -> None:
    required_major: int = 3
    required_minor: int = 11
    current_version: tuple[int, int, int] = sys.version_info[:3]

    if current_version[0] != required_major or current_version[1] != required_minor:
        raise RuntimeError(
            f"Python {required_major}.{required_minor}.xx is required, "
            f"but you are using {current_version[0]}.{current_version[1]}.{current_version[2]}"
        )


check_python_version()

In [ ]:
# import important liberies Important!! usse  python 3.11 or below
import pandas as pd
import pyreadr
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
from tabulate import tabulate
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
# Converting .rData into dataframe
fault_free_training_dict = pyreadr.read_r("data/TEP_FaultFree_Training.RData")
fault_free_testing_dict = pyreadr.read_r("data/TEP_FaultFree_Testing.RData")

faulty_training_dict = pyreadr.read_r("data/TEP_Faulty_Training.RData")
faulty_testing_dict = pyreadr.read_r("data/TEP_Faulty_Testing.RData")

In [ ]:
DF_FF_TRAINING_RAW = fault_free_training_dict["fault_free_training"]
DF_FF_TEST_RAW = fault_free_testing_dict["fault_free_testing"]

DF_F_TRAINING_RAW = faulty_training_dict["faulty_training"]
DF_F_TEST_RAW = faulty_testing_dict["faulty_testing"]

# Global Configuration

In [ ]:
VERSION = "1.00"
OUTPUT_PATH="output"

In [ ]:
# Model Names to use in plots and tables

XGBOOST = "XGBoost"
RANDOM_FOREST = "Random Forest"
NEURAL_NET= "Neural Net"

# Custom functions

## Compute_first_detection_delay

In [ ]:
# Compute_first_detection_delay
import numpy as np
import pandas as pd
from typing import Union, List


def compute_first_detection_delay(
        y_true: Union[List[int], np.ndarray],
        y_pred: Union[List[int], np.ndarray]) -> pd.DataFrame:
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    fault_labels = sorted(np.unique(y_true[y_true > 0]))
    results = []

    for fault in fault_labels:
        fault_mask = y_true == fault
        fault_indices = np.where(fault_mask)[0]  # Get indices of fault samples
        if len(fault_indices) == 0:
            delay = None
            print("No fault samples.")
        else:
            start_index: int = fault_indices[
                0]  # Get the first index of the fault
            detection_indices = np.where(
                (y_pred == fault) & (np.arange(len(y_pred)) >= start_index)
            )[0]  # Get indices where the prediction matches the fault and is after the start index
            delay = (detection_indices[0] -
                     start_index if len(detection_indices) > 0 else None)

        results.append({
            "Fault":
            fault,
            "First Detection Delay":
            delay if (delay is not None and delay >= 0) else np.nan,
        })

    return pd.DataFrame(results)


## Data export functions

In [ ]:
import os
from datetime import datetime
import matplotlib.pyplot as plt

VERSION: str = VERSION if "VERSION" in globals() and VERSION else "default"
OUTPUT_PATH: str = OUTPUT_PATH if "OUTPUT_PATH" in globals(
) and OUTPUT_PATH else "output"


def save_plot(plot_name: str, suffix: str = "", plot_path:str="default") -> None:
    """
    Save current matplotlib figure with a structured filename.

    Args:
        plot_name (str): Base name for the plot.
        suffix (str): Optional suffix, e.g., class ID or 'average'.
    """
    #timestamp: str = datetime.now().strftime("%S")
    timestamp: str = ""
    base_dir: str = os.path.join(OUTPUT_PATH, VERSION, plot_path)
    os.makedirs(base_dir, exist_ok=True)

    filename: str = f"{plot_name}_{suffix}_v{VERSION}_{timestamp}.png" if suffix else f"{plot_name}_v{VERSION}_{timestamp}.png"
    filepath: str = os.path.join(base_dir, filename)

    plt.savefig(filepath, bbox_inches="tight")
    #plt.show()

In [ ]:
def save_dataframe(df: pd.DataFrame, name: str, suffix: str = "") -> None:
    """
    Save a DataFrame to CSV with versioned and timestamped filename.

    Args:
        df (pd.DataFrame): DataFrame to save.
        name (str): Base name for the file.
        suffix (str): Optional suffix (e.g. class id, 'summary').
    """
    #timestamp: str = datetime.now().strftime("%S")
    timestamp: str = ""
    base_dir: str = os.path.join(OUTPUT_PATH, VERSION)
    os.makedirs(base_dir, exist_ok=True)

    filename: str = f"{name}_{suffix}_v{VERSION}_{timestamp}.csv" if suffix else f"{name}_v{VERSION}_{timestamp}.csv"
    filepath: str = os.path.join(base_dir, filename)

    df.to_csv(filepath, index=True)

# Prepare data

In [ ]:
TARGET_VARIABLE_COLUMN_NAME = "faultNumber"
SIMULATION_RUN_COLUMN_NAME = "simulationRun"
COLUMNS_TO_REMOVE = ["simulationRun", "sample"]
SKIPED_FAULTS = []
FAULTS_TO_BE_MERGED_TOGETHER = [3, 8,9,18, 15]
MERGE_FAUTS_TO_NUMBER = 3
FAULT_INJECTION_STARTING_POINT = 25

DF_F_TRAIN_SKIPPED_FAULTS = DF_F_TRAINING_RAW[~DF_F_TRAINING_RAW[TARGET_VARIABLE_COLUMN_NAME].isin(SKIPED_FAULTS)].reset_index(drop=True)
DF_F_TEST_SKIPPED_FAULTS = DF_F_TEST_RAW[~DF_F_TEST_RAW[TARGET_VARIABLE_COLUMN_NAME].isin(SKIPED_FAULTS)].reset_index(drop=True)

# **Reduce Training and test data for simplicity during development and testing**
# **** THE DATA SHOULD STAY BALANCED !!!! *****

# reduce training data
DF_FF_TRAINING_REDUCED = DF_FF_TRAINING_RAW[(DF_FF_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] > 0) & (DF_FF_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] < 2)].drop(columns=COLUMNS_TO_REMOVE, axis=1)
DF_F_TRAINING_REDUCED = DF_F_TRAIN_SKIPPED_FAULTS[(DF_F_TRAIN_SKIPPED_FAULTS[SIMULATION_RUN_COLUMN_NAME] > 4 )& (DF_F_TRAIN_SKIPPED_FAULTS[SIMULATION_RUN_COLUMN_NAME] < 6) &(DF_F_TRAIN_SKIPPED_FAULTS["sample"] > FAULT_INJECTION_STARTING_POINT)].drop(columns=COLUMNS_TO_REMOVE, axis=1)

# reduce test data
DF_FF_TEST_REDUCED = DF_FF_TEST_RAW[(DF_FF_TEST_RAW[SIMULATION_RUN_COLUMN_NAME] > 2) & (DF_FF_TEST_RAW[SIMULATION_RUN_COLUMN_NAME] < 4)].drop(columns=COLUMNS_TO_REMOVE, axis=1)
DF_F_TEST_REDUCED = DF_F_TEST_SKIPPED_FAULTS[(DF_F_TEST_SKIPPED_FAULTS[SIMULATION_RUN_COLUMN_NAME] > 5)& (DF_F_TEST_SKIPPED_FAULTS[SIMULATION_RUN_COLUMN_NAME] < 7) &(DF_F_TEST_SKIPPED_FAULTS["sample"] > FAULT_INJECTION_STARTING_POINT)].drop(columns=COLUMNS_TO_REMOVE, axis=1)

DF_F_TRAINING_REDUCED.head()


In [ ]:
def check_balance_difference(df1: pd.DataFrame, df2: pd.DataFrame, threshold: int = 100) -> None:
    size_diff: int = abs(df1.shape[0] - df2.shape[0])
    print("Data differen is: ", size_diff)
    if size_diff > threshold:
        raise ValueError(f"Data imbalance too large: difference = {size_diff} rows")


check_balance_difference(DF_FF_TRAINING_REDUCED,DF_F_TRAINING_REDUCED.query("faultNumber == 1"))
check_balance_difference(DF_FF_TEST_REDUCED,DF_F_TEST_REDUCED.query("faultNumber == 1"))


In [ ]:
print(DF_FF_TRAINING_REDUCED.shape)
print(DF_F_TRAINING_REDUCED.query("faultNumber == 1").shape) # the output of this line should be close to df_ff_training_reduced.shape, so the data is balanced

In [ ]:
# Prepare data for Supervised training and testing

DF_TRAINING_REDUCED_CONCATED = pd.concat([DF_FF_TRAINING_REDUCED, DF_F_TRAINING_REDUCED])
DF_TEST_REDUCED_CONCATED = pd.concat([DF_FF_TEST_REDUCED, DF_F_TEST_REDUCED])


# Standardize the data: It centers the data around 0 and scales it based on standard deviation.
sc = StandardScaler()
sc.fit(DF_TRAINING_REDUCED_CONCATED.drop(columns=[TARGET_VARIABLE_COLUMN_NAME],axis=1))
X_TRAIN = sc.transform(DF_TRAINING_REDUCED_CONCATED.drop(columns=[TARGET_VARIABLE_COLUMN_NAME],axis=1))
Y_TRAIN_DF = DF_TRAINING_REDUCED_CONCATED[TARGET_VARIABLE_COLUMN_NAME]

# Encode the target variable: -LabelEncoder() Takes a list/array of categorical labels (e.g., ['shift', 'trend', 'none']), -Assigns a unique integer to each category (e.g., ['none' → 0, 'shift' → 1, 'trend' → 2]), -Returns a NumPy array of integers corresponding to the original labels
le = LabelEncoder()
Y_TRAIN = le.fit_transform(Y_TRAIN_DF)

# Set the features and target variable for testing
X_TEST_REDUCED = sc.transform(DF_TEST_REDUCED_CONCATED.drop(columns=[TARGET_VARIABLE_COLUMN_NAME], axis=1))
Y_TEST_REDUCED_DF = DF_TEST_REDUCED_CONCATED[TARGET_VARIABLE_COLUMN_NAME]

# Encode the target variable for testing
Y_TEST_REDUCED = le.fit_transform(Y_TEST_REDUCED_DF)

# One-hot encode the target variable
encoder_1 = OneHotEncoder(sparse_output=False)
Y_reshabed = (DF_TRAINING_REDUCED_CONCATED[TARGET_VARIABLE_COLUMN_NAME].to_numpy().reshape(-1, 1))
# .fit_transform() fits the encoder to the data and then transforms it.
# Use this on the training data to learn the encoding mapping and apply it.
Y_ENC_TRAIN = encoder_1.fit_transform(Y_reshabed)

# .transform() only applies the learned encoding to new data.
# Use this on test data to encode using the mapping learned from training data.
Y_test_reshabed = (DF_TEST_REDUCED_CONCATED[TARGET_VARIABLE_COLUMN_NAME].to_numpy().reshape(-1, 1))
Y_ENC_TEST_REDUCED = encoder_1.transform(Y_test_reshabed)

print(
    "Training Data with the following Fault Numbers:",
    DF_TEST_REDUCED_CONCATED[TARGET_VARIABLE_COLUMN_NAME].unique(),
)
DF_TRAINING_REDUCED_CONCATED.head()

In [ ]:
# Count the number of samples in each class
class_counts = DF_TEST_REDUCED_CONCATED[
    TARGET_VARIABLE_COLUMN_NAME].value_counts()
print("Class counts in training data:")
print(class_counts)
# summe all samples
print("Total samples in training data:", class_counts.sum())

In [ ]:
# Standardize the data Unsupervised learning

# TRAIN data

# X
X_INCONTROL_TRAIN_REDUCED_DF = DF_FF_TRAINING_REDUCED.drop(columns=[TARGET_VARIABLE_COLUMN_NAME], axis=1)
sc = StandardScaler()
sc.fit(X_INCONTROL_TRAIN_REDUCED_DF)
X_INCONTROL_TRAIN_REDUCED = sc.transform(X_INCONTROL_TRAIN_REDUCED_DF)

# Y
Y_TRAIN_ANOMALY_REDUCED_DF = DF_TRAINING_REDUCED_CONCATED[TARGET_VARIABLE_COLUMN_NAME].apply(lambda x: 0 if x == 0 else 1) ## change target variable to only 2 classes: 0 and 1: 0 is in control , bigger than 0 is faulty
encoder_2 = OneHotEncoder(sparse_output=False)
Y_reshabed = Y_TRAIN_ANOMALY_REDUCED_DF.to_numpy().reshape(-1, 1)
Y_ENC_ANOMALY_TRAIN_REDUCED = encoder_2.fit_transform(Y_reshabed)



# Test data

# X
X_INCONTROL_TEST_REDUCED_DF = DF_FF_TEST_REDUCED.drop( columns=[TARGET_VARIABLE_COLUMN_NAME], axis=1)
sc = StandardScaler()
sc.fit(X_INCONTROL_TEST_REDUCED_DF)
X_INCONTROL_TEST_REDUCED = sc.transform(X_INCONTROL_TEST_REDUCED_DF)

X_OUT_OF_CONTROL_TEST_REDUCED_DF = DF_F_TEST_REDUCED.drop(columns=[TARGET_VARIABLE_COLUMN_NAME], axis=1)
sc = StandardScaler()
sc.fit(X_OUT_OF_CONTROL_TEST_REDUCED_DF)
X_OUT_OF_CONTROL_TEST_REDUCED = sc.transform(X_OUT_OF_CONTROL_TEST_REDUCED_DF)

# Y
Y_TEST_ANOMALY_REDUCED_DF = DF_TEST_REDUCED_CONCATED[TARGET_VARIABLE_COLUMN_NAME].apply(lambda x: 0 if x == 0 else 1) ## change target variable to only 2 classes: 0 and 1: 0 is in control , bigger than 0 is faulty
Y_test_reshabed = Y_TEST_ANOMALY_REDUCED_DF.to_numpy().reshape(-1, 1)
Y_ENC_ANOMALY_TEST_REDUCED = encoder_2.transform(Y_test_reshabed)

y_test_binary = Y_test_reshabed.ravel().tolist()

In [ ]:
# Merge hard faut together to a new faut, like faut 3,5,15 to new faut 3 or some other number like 40

# Y Train
Y_TRAIN_MERGED_FAULTS_REDUCED_DF = DF_TRAINING_REDUCED_CONCATED[TARGET_VARIABLE_COLUMN_NAME].apply(lambda x: 3 if x in FAULTS_TO_BE_MERGED_TOGETHER else x)
encoder_3 = OneHotEncoder(sparse_output=False)
Y_reshabed_train = Y_TRAIN_MERGED_FAULTS_REDUCED_DF.to_numpy().reshape(-1, 1)
Y_ENC_MERGED_TRAIN_REDUCED = encoder_3.fit_transform(Y_reshabed_train)

# Y Test
Y_TEST_MERGED_FAULTS_DF = DF_TEST_REDUCED_CONCATED[TARGET_VARIABLE_COLUMN_NAME].apply(lambda x: 3 if x in FAULTS_TO_BE_MERGED_TOGETHER else x)
Y_reshabed_test = Y_TEST_MERGED_FAULTS_DF.to_numpy().reshape(-1, 1)
Y_ENC_MERGED_TEST_REDUCED = encoder_3.transform(Y_reshabed_test)

# The output should not have the merged fault, like 15, it is merged to 3, or what ever it is configured for
print(np.unique(Y_reshabed_train))
print(np.unique(Y_reshabed_test))


In [ ]:
DF_F_TEST_REDUCED.query("faultNumber == 2").shape


In [ ]:
# Standardize the data for EDA

# Train
sc = StandardScaler()
sc.fit(DF_F_TRAINING_RAW.drop(columns=[TARGET_VARIABLE_COLUMN_NAME], axis=1))
X_F_TRAIN = sc.transform(DF_F_TRAINING_RAW.drop(columns=[TARGET_VARIABLE_COLUMN_NAME], axis=1))

Y_F_TRAIN = DF_F_TRAINING_RAW[TARGET_VARIABLE_COLUMN_NAME].to_numpy()
le = LabelEncoder()
Y_F_TRAIN = le.fit_transform(Y_F_TRAIN)

# Test
sc = StandardScaler()
sc.fit(DF_FF_TRAINING_RAW.drop(columns=[TARGET_VARIABLE_COLUMN_NAME], axis=1))
X_FF_TRAIN = sc.transform(DF_FF_TRAINING_RAW.drop(columns=[TARGET_VARIABLE_COLUMN_NAME], axis=1))
Y_FF_TRAIN = DF_FF_TRAINING_RAW[TARGET_VARIABLE_COLUMN_NAME].to_numpy()
le = LabelEncoder()
Y_FF_TRAIN = le.fit_transform(Y_FF_TRAIN)

In [ ]:
DF_F_TRAINING_RAW.head()

# Exploratory Data Analysis (EDA)

## Metadata overview

**The TEP variables (columns 4 to 55) were sampled every 3 minutes for a total duration of 25 hours and 48 hours respectively.
The faults were introduced 1 hour into the Faulty Training
and 8 hours into Faulty Testing datasets**

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.stats import skew, kurtosis
from typing import Optional


def calculate_statistics_per_fault_run(
        df_faulty: pd.DataFrame, df_normal: pd.DataFrame) -> pd.DataFrame:
    base_dir: str = os.path.join(OUTPUT_PATH, VERSION, "fault_statistics.csv")
    output_path: Path = Path(base_dir)
    output_path.parent.mkdir(parents=True, exist_ok=True)

    if output_path.exists():
        return pd.read_csv(output_path)

    features: list[str] = [
        col for col in df_faulty.columns
        if col not in [TARGET_VARIABLE_COLUMN_NAME, SIMULATION_RUN_COLUMN_NAME, "time", "sample"]
    ]

    def compute_stats(df: pd.DataFrame) -> pd.DataFrame:
        rows: list[dict] = []
        grouped = df.groupby([TARGET_VARIABLE_COLUMN_NAME, SIMULATION_RUN_COLUMN_NAME])
        for (fault, run), group in grouped:
            for feature in features:
                values: pd.Series = group[feature].dropna()
                if values.empty:
                    continue
                rows.append({
                    TARGET_VARIABLE_COLUMN_NAME:
                    fault,
                    SIMULATION_RUN_COLUMN_NAME:
                    run,
                    "feature":
                    feature,
                    "mean":
                    values.mean(),
                    "std":
                    values.std(),
                    "skewness":
                    skew(values),
                    "kurtosis":
                    kurtosis(values),
                    "autocorr_1":
                    values.autocorr(lag=1) if len(values) > 1 else np.nan,
                })
        return pd.DataFrame(rows)

    df_all: pd.DataFrame = pd.concat([df_normal, df_faulty], ignore_index=True)
    result_df: pd.DataFrame = compute_stats(df_all)
    result_df.to_csv(output_path, index=False)
    save_dataframe(df=result_df, name="fault_statistics")
    return result_df


# Reduce data for speed
statics_f_df = DF_F_TRAINING_RAW[(DF_F_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] >= 1)
                             & (DF_F_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] < 4)]
statics_ff_df = DF_FF_TRAINING_RAW[(DF_FF_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] >= 1)
                               & (DF_FF_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] < 4)]

stats_df = calculate_statistics_per_fault_run(df_faulty=statics_f_df,
                                              df_normal=statics_ff_df)

In [ ]:
stats_df.query("faultNumber == 3 and simulationRun == 1")

In [ ]:
stats_df.head()

In [ ]:
features_fault3 = set(
    stats_df.query("faultNumber == 3 and simulationRun == 1")["feature"])
features_fault0 = set(
    stats_df.query("faultNumber == 0 and simulationRun == 1")["feature"])

missing_in_0 = features_fault3 - features_fault0
missing_in_3 = features_fault0 - features_fault3

print("Missing in fault 0:", missing_in_0)
print("Missing in fault 3:", missing_in_3)


## Fault injection point analysis

## Boxplots

In [ ]:
# Plotting boxplots for all features

faultNumber = 3
simulationRun = 10
df_selected_f_data = DF_F_TRAINING_RAW[
    (DF_F_TRAINING_RAW[TARGET_VARIABLE_COLUMN_NAME] == faultNumber)
    & (DF_F_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] == simulationRun)]
df_select_ff_data = DF_FF_TRAINING_RAW[
    (DF_FF_TRAINING_RAW[TARGET_VARIABLE_COLUMN_NAME] == 0)
    & (DF_FF_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] == simulationRun)]

used_data = df_select_ff_data

feature_columns = DF_FF_TRAINING_RAW.columns[3:]
num_features = len(feature_columns)

# Define the number of rows and columns for the subplots in the grid
num_cols = min(4, num_features)  # Maximum of 4 columns
num_rows = int(np.ceil(num_features / num_cols))
fig, axes = plt.subplots(num_rows,
                         num_cols,
                         figsize=(4 * num_cols, 3 * num_rows))
fig.suptitle("Boxplots of All Features", fontsize=16, y=1.02)

for i, col in enumerate(feature_columns):
    row_index = i // num_cols
    col_index = i % num_cols
    ax = axes[row_index, col_index]

    ax.boxplot(
        used_data[col],
        patch_artist=True,
        boxprops=dict(facecolor="lightblue", color="navy"),
        medianprops=dict(color="red"),
        whiskerprops=dict(color="gray"),
        capprops=dict(color="gray"),
        flierprops=dict(marker="o", color="darkorange", alpha=0.5),
    )

    ax.grid(True, linestyle="--", alpha=0.5)
    ax.set_ylabel(col)
    # ax.set_xticklabels([col], rotation=45)
    ax.set_title(col.replace("_", " "))

for i in range(num_features, num_rows * num_cols):
    fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.show()

## Distribution Plots

In [ ]:
# Plotting distribution of selected features in f and ff beside each other to compare fault vs normal

faultNumber = 3
simulationRun = 1
df_selected_f_data = DF_F_TRAINING_RAW[
    (DF_F_TRAINING_RAW[TARGET_VARIABLE_COLUMN_NAME] == faultNumber)
    & (DF_F_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] == simulationRun)]
df_select_ff_data = DF_FF_TRAINING_RAW[
    (DF_FF_TRAINING_RAW[TARGET_VARIABLE_COLUMN_NAME] == 0)
    & (DF_FF_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] == simulationRun)]

# plotting distribution of selected features f and ff beside each other to compare fault vs normal
# selected features to plot
# selected_features = ['xmeas_1', 'xmeas_2']
selected_features = df_selected_f_data.columns[
    3:]  # Select features from index 3 to the end
# Create a figure with subplots for each selected feature
fig, axes = plt.subplots(len(selected_features),
                         2,
                         figsize=(12, 6 * len(selected_features)))
for i, feature in enumerate(selected_features):
    # Plotting the faulty data gaussian distribution
    sns.histplot(
        df_selected_f_data[feature],
        kde=True,
        ax=axes[i, 0],
        color="red",
        label="Faulty Data",
    )
    axes[i, 0].set_title(f"Faulty Data - {feature}")
    axes[i, 0].set_xlabel(feature)
    axes[i, 0].set_ylabel("Density")
    axes[i, 0].legend()
    # Plotting the fault-free data gaussian distribution
    sns.histplot(
        df_select_ff_data[feature],
        kde=True,
        ax=axes[i, 1],
        color="blue",
        label="Fault-Free Data",
    )
    axes[i, 1].set_title(f"Fault-Free Data - {feature}")
    axes[i, 1].set_xlabel(feature)
    axes[i, 1].set_ylabel("Density")
    axes[i, 1].legend()
plt.tight_layout()
plt.show()

## Visualize class separation

In [ ]:
# Visualize class separation and structure in high-dimensional process data using t-SNE embedding to 2D.

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns


def plot_tsne_visualization(x_f_train: np.ndarray,
                            y_f_labeled_train: np.ndarray,
                            step: int = 50) -> None:
    """
    Visualize class separation and structure in high-dimensional process data using t-SNE embedding to 2D.

    Input:
        x_f_train: High-dimensional feature matrix (e.g., 54 dimensions).
        y_f_labeled_train: 1D label array corresponding to x_f_train.
        step: Downsampling factor to reduce computational load (default=50).

    Output:
        A 2D scatter plot showing t-SNE projection colored and styled by labels.
    """

    # Downsample the data to reduce computation
    x_down = x_f_train[::step, :]
    y_label = y_f_labeled_train[::step]

    # Apply t-SNE to project high-dimensional data into 2D space
    x_embedded = TSNE(n_components=2, learning_rate="auto",
                      init="random").fit_transform(x_down)

    # Create a scatter plot of the 2D embedded data
    f, ax = plt.subplots(figsize=(10, 6))
    sns.scatterplot(
        x=x_embedded[:, 0],
        y=x_embedded[:, 1],
        hue=y_label,
        style=y_label,
        palette="bright",
        edgecolor="black",
    )
    plt.legend(bbox_to_anchor=(1.1, 1.05))
    plt.title("t-SNE Visualization of Labeled Data")
    plt.show()


# plot_tsne_visualization(x_f_train, y_f_labeled_train)  # Uncomment to run t-SNE visualization (heavy- take time)
plot_tsne_visualization(
    X_TRAIN,
    Y_TRAIN_DF)  # Uncomment to run t-SNE visualization (heavy- take time)


In [ ]:
# scatter plot of the first two features of the training data
plt.figure(figsize=(10, 6))
plt.scatter(X_TRAIN[:, 0],
            X_TRAIN[:, 1],
            c=Y_TRAIN_DF,
            cmap="viridis",
            alpha=0.5)
plt.colorbar(label="Fault Number")
plt.title("Scatter Plot of First Two Features of Training Data")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.grid()
plt.show()

## play

In [ ]:
# plot distribution gaussian distribution of the first two features of the training data
plt.figure(figsize=(12, 6))
sns.histplot(X_TRAIN[:, 0], kde=True, color="blue", label="Feature 1")
sns.histplot(X_TRAIN[:, 1], kde=True, color="orange", label="Feature 2")
plt.title("Gaussian Distribution of First Two Features of Training Data")
plt.xlabel("Feature Value")
plt.ylabel("Density")
plt.legend()
plt.grid()
plt.show()

## Fault injection point analysis

In [ ]:
# Fault injection point analysis: its started at sample  almost 25
feature = "xmeas_3"  # Example feature to inspect
fault_number = 3  # Example fault number to inspect
simulationRun = 6
feature_a = DF_F_TRAINING_RAW.query(
    "faultNumber == @fault_number and simulationRun == @simulationRun"
)[feature].reset_index(drop=True)
feature_b = DF_FF_TRAINING_RAW.query(
    "simulationRun == @simulationRun")[feature].reset_index(drop=True)

delta = feature_a - feature_b

# print(delta)
plt.figure(figsize=(12, 6))
plt.plot(delta[:50], label="Delta (feature A - feature B)", color="red")
plt.axhline(0, color="black", linestyle="--", linewidth=1)
plt.title(
    f"Delta for {feature} between Fault A and Fault B for same simulation run (same seed)"
)
plt.xlabel("Sample Index")
plt.ylabel("Delta Value")
plt.legend()
plt.grid()
plt.show()


def compare_faults_stat(
    stats_df: pd.DataFrame,
    fault_a: int,
    fault_b: int,
    run_id: int,
    stat: str = "mean",
) -> pd.DataFrame:
    df_a = stats_df.query(
        "faultNumber == @fault_a and simulationRun == @run_id")[[
            "feature", stat
        ]].rename(columns={stat: f"fault_{fault_a}"})

    df_b = stats_df.query(
        "faultNumber == @fault_b and simulationRun == @run_id")[[
            "feature", stat
        ]].rename(columns={stat: f"fault_{fault_b}"})

    merged = pd.merge(df_a, df_b, on="feature", how="outer")
    merged["delta"] = merged[f"fault_{fault_a}"] - merged[f"fault_{fault_b}"]
    return merged


compare_df = compare_faults_stat(stats_df,
                                 fault_a=3,
                                 fault_b=0,
                                 run_id=1,
                                 stat="mean")

compare_df.head()

stats_df.feature.unique()

des = DF_FF_TRAINING_RAW.iloc[:, 3:].describe()
cal = des.iloc[1:3, :].T
cal["variance"] = cal["std"]**2
print(cal.head())


In [ ]:
# Determine the fault insection data point

# Determine the fault insection data point

# This function will plot the feature time series before and after fault injection
# It will also mark the fault injection point and show median values before and after the injection.
# It is useful for visualizing how features change around the time a fault is injected.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def plot_fault_injection_segment(
    df: pd.DataFrame,
    fault_injection_index: int,
    features: list[str],
    fault_number: int,
    point_start: int = 0,
    point_end: int = 500,
) -> None:
    """
    Plot feature time series before and after fault injection.

    Parameters:
    - df: DataFrame containing time series (1 simulation run).
    - fault_injection_index: Time step where fault is injected.
    - features: List of feature names to plot.
    - window: Number of time steps before and after injection to show.
    """
    # Filter only the rows with the selected fault number
    df: pd.DataFrame = df[df[TARGET_VARIABLE_COLUMN_NAME] == fault_number].reset_index(
        drop=True)

    x_range: np.ndarray = np.arange(point_start, point_end)

    df_window: pd.DataFrame = df.iloc[point_start:point_end]

    n_features: int = len(features)
    fig, axes = plt.subplots(n_features,
                             1,
                             figsize=(30, 3 * n_features),
                             sharex=True)

    if n_features == 1:
        axes = [axes]

    for i, feature in enumerate(features):
        ax = axes[i]
        y = df_window[feature].values
        ax.plot(x_range, y, label=feature, color="black")

        # Mark every 5th time index
        for x in x_range:
            if x % 5 == 0:
                ax.axvline(x, color="gray", linestyle=":", linewidth=0.5)
                ax.text(
                    x,
                    ax.get_ylim()[1],
                    f"{x}",
                    color="gray",
                    fontsize=8,
                    ha="center",
                    va="bottom",
                    rotation=90,
                    backgroundcolor="white",
                )

        # Vertical fault injection marker
        ax.axvline(
            fault_injection_index,
            color="red",
            linestyle="--",
            linewidth=2,
            label="Fault Injected",
        )

        # Text showing the time index
        ax.text(
            fault_injection_index,
            ax.get_ylim()[1],
            f"t={fault_injection_index}",
            color="red",
            fontsize=10,
            ha="center",
            va="bottom",
            rotation=0,
            backgroundcolor="white",
        )

        # Median before/after
        before = df.loc[point_start:fault_injection_index - 1, feature]
        after = df.loc[fault_injection_index:point_end - 1, feature]
        before_med = np.median(before)
        after_med = np.median(after)

        ax.axhline(before_med,
                   color="blue",
                   linestyle=":",
                   label="Median Before")
        ax.axhline(after_med,
                   color="green",
                   linestyle=":",
                   label="Median After")

        ax.set_ylabel(feature)
        ax.grid(True)

        if i == 0:
            ax.legend(loc="best")

    plt.xlabel("Time Index")
    plt.suptitle("Feature Response Around Fault Injection", fontsize=14)
    plt.tight_layout()
    plt.show()


df_fault = DF_F_TRAINING_RAW[(DF_F_TRAINING_RAW["simulationRun"] == 2.0)].reset_index(
    drop=True)

print(len(df_fault))

plot_fault_injection_segment(
    df=df_fault,
    fault_injection_index=FAULT_INJECTION_STARTING_POINT,
    # features=['xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6'],
    features=df_fault.columns[3:9],
    fault_number=1,
    point_start=1,
    point_end=500,
)

In [ ]:
# Compare fault vs normal time series for selected features.
# This help to caputre the differences between faulty and normal segments of the time series data and determine the start of the fault injection.

simulationRun = 1.0
fault_number = 3
df_fault = DF_F_TRAINING_RAW[(
    DF_F_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] == simulationRun)].reset_index(drop=True)
df_normal = DF_FF_TRAINING_RAW[(
    DF_FF_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] == simulationRun)].reset_index(drop=True)

# Compare fault vs normal time series for selected features.
# This help to caputre the differences between faulty and normal segments of the time series data and determine the start of the fault injection.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def plot_fault_vs_normal_segment(
    df_fault: pd.DataFrame,
    df_normal: pd.DataFrame,
    features: list[str],
    point_start: int,
    point_end: int,
    fault_number: int,
    fault_injection_index: int = 160,
) -> None:
    """
    Compare fault vs normal time series for selected features.

    Parameters:
    - df_fault: DataFrame with faults (should include faultNumber column).
    - df_normal: Normal (non-faulty) baseline DataFrame.
    - features: List of feature names to compare.
    - point_start: Start index of window.
    - point_end: End index of window (exclusive).
    - fault_number: Fault number to extract from df_fault.
    """
    df_fault = df_fault[df_fault[TARGET_VARIABLE_COLUMN_NAME] == fault_number].reset_index(
        drop=True)
    df_fault_window = df_fault.iloc[point_start:point_end]
    df_normal_window = df_normal.iloc[point_start:point_end]

    x_range = np.arange(point_start, point_end)
    n_features = len(features)
    fig, axes = plt.subplots(n_features,
                             1,
                             figsize=(30, 3 * n_features),
                             sharex=True)

    if n_features == 1:
        axes = [axes]

    for i, feature in enumerate(features):
        ax = axes[i]

        y_fault = df_fault_window[feature].values
        y_normal = df_normal_window[feature].values

        ax.plot(x_range, y_fault, label=f"Fault {fault_number}", color="red")
        ax.plot(x_range, y_normal, label="Normal", color="black")

        # Median lines
        ax.axhline(
            np.median(y_normal),
            color="black",
            linestyle="--",
            label="Normal Median",
        )
        ax.axhline(
            np.median(y_fault),
            color="red",
            linestyle="--",
            label="Fault Median",
        )
        ax.axvline(
            fault_injection_index,
            color="green",
            linestyle="--",
            linewidth=2,
            label="Fault Injected",
        )
        # Mark every 5th index
        for x in x_range:
            if x % 5 == 0:
                ax.axvline(x, color="gray", linestyle=":", linewidth=0.5)
                ax.text(
                    x,
                    ax.get_ylim()[1],
                    f"{x}",
                    color="gray",
                    fontsize=8,
                    ha="center",
                    va="bottom",
                    rotation=90,
                    backgroundcolor="white",
                )
        ax.set_ylabel(feature)
        ax.grid(True)
        if i == 0:
            ax.legend(loc="best")
    # Place suptitle before tight_layout to ensure it appears above the plots
    # plt.suptitle(f"Comparison of Fault {fault_number} vs Normal", fontsize=14)
    plt.xlabel("Time Index")

    plt.tight_layout()
    plt.show()


plot_fault_vs_normal_segment(
    df_fault=df_fault,
    df_normal=df_normal,
    features=DF_F_TRAINING_RAW.
    columns[3:],  # Select all features, which strats from index 3
    point_start=20,
    point_end=50,
    fault_number=fault_number,
    fault_injection_index=FAULT_INJECTION_STARTING_POINT,
)


## Time Serie Plots

In [ ]:
# Plotting Training Data for Simulation Run 1 and Fault Number 0
df_data_to_plot = DF_FF_TRAINING_RAW[(DF_FF_TRAINING_RAW[TARGET_VARIABLE_COLUMN_NAME] == 0)
                                 & (DF_FF_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] == 1)]

feature_columns = df_normal.columns[3:]
num_features = len(feature_columns)

# Define the number of rows and columns for the subplots
num_cols = min(4, num_features)  # Maximum of 4 columns
num_rows = int(np.ceil(num_features / num_cols))
size_multiplier = 2  # Adjust this value to change the size of the subplots
# Create subplots
fig, axes = plt.subplots(
    num_rows,
    num_cols,
    figsize=(4 * num_cols * size_multiplier, 3 * num_rows * size_multiplier),
)

for i, col in enumerate(feature_columns):
    # Determine the row and column index (position) for the subplot in the grid
    row_index = i // num_cols
    col_index = i % num_cols
    ax = axes[row_index, col_index]

    ax.plot(df_data_to_plot["sample"], df_data_to_plot[col])
    ax.set_xlabel("sample no.")
    ax.set_ylabel(col)
    ax.set_title(col.replace("_", " "))

for i in range(num_features, num_rows * num_cols):
    fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.show()


In [ ]:
# Plotting a selected feature against sample numbers for multiple simulation runs and no faults

# selected column to plot # Uncomment the column you want to plot
# col = 'A_feed_stream'
col = "xmeas_1"

# Define the simulation runs to plot
simRuns = np.arange(1, 500, 50)
num_simRuns = len(simRuns)

# Define the number of rows and columns for the subplots in the grid
num_cols = min(3, num_simRuns)  # Maximum of 2 columns
num_rows = int(np.ceil(num_simRuns / num_cols))
size_multiplier = 2  # Adjust this value to change the size of the subplots
# Create subplots
fig, axes = plt.subplots(
    num_rows,
    num_cols,
    figsize=(4 * num_cols * size_multiplier, 3 * num_rows * size_multiplier),
)

for i, simRun in enumerate(simRuns):
    pp = DF_FF_TRAINING_RAW[(DF_FF_TRAINING_RAW[TARGET_VARIABLE_COLUMN_NAME] == 0)
                        & (DF_FF_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] == simRun)]
    row_index = i // num_cols
    col_index = i % num_cols
    ax = axes[row_index, col_index]
    ax.plot(pp["sample"], pp[col])
    ax.set_xlabel("sample no.")
    ax.set_ylabel(col)
    ax.set_title(f"Simulation run - {simRun}")

# Remove any empty subplots
for i in range(num_simRuns, num_rows * num_cols):
    fig.delaxes(axes.flatten()[i])

plt.tight_layout()

plt.show()

## Correlation matrix

In [ ]:
# Plotting the correlation matrix of the features

# Load the data into a pandas DataFrame
data = DF_FF_TRAINING_RAW[DF_FF_TRAINING_RAW["simulationRun"] == 1].iloc[:, 3:]

# Calculate the correlation matrix
corr = data.corr()

# Create a heatmap with annotations
sns.set_theme(style="white")
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(50, 50))
sns.heatmap(
    corr,
    mask=mask,
    cmap="coolwarm",
    annot=True,
    fmt=".2f",
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
)
plt.show()

In [ ]:
# Plotting the correlation matrix with a threshold
# Set correlation threshold
threshold: float = 0.95

# Load the data into a pandas DataFrame
data: pd.DataFrame = DF_FF_TRAINING_RAW[DF_FF_TRAINING_RAW[SIMULATION_RUN_COLUMN_NAME] ==
                                    1].iloc[:, 3:]

# Calculate correlation matrix
corr: pd.DataFrame = data.corr()

# Remove self-correlation (diagonal only)
corr_abs: pd.DataFrame = corr.abs().copy()
np.fill_diagonal(corr_abs.values, 0.0)

# Identify features with any correlation above threshold
selected_features: set[str] = set(corr_abs.columns[(corr_abs
                                                    > threshold).any()])
selected_features_list: list[str] = sorted(selected_features)

# Filter correlation matrix to only those features
filtered_corr: pd.DataFrame = corr.loc[selected_features_list,
                                       selected_features_list]

# Plot heatmap
sns.set_theme(style="white")
mask: np.ndarray = np.triu(np.ones_like(filtered_corr, dtype=bool))

f, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(
    filtered_corr,
    mask=mask,
    cmap="coolwarm",
    annot=True,
    fmt=".2f",
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
)
plt.title(f"Correlation Matrix (|corr| > {threshold})")
plt.tight_layout()
plt.show()


## Plotting high-correlation pairs with interactive widgets as Time Series

In [ ]:
# Plotting high-correlation pairs with interactive widgets as Time Series
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox, Output, Dropdown

# 1. Compute correlation matrix and extract high-correlation pairs
data: pd.DataFrame = DF_FF_TRAINING_RAW[DF_FF_TRAINING_RAW["simulationRun"] ==
                                    1].iloc[:, 3:]
corr: pd.DataFrame = data.corr()

# Zero diagonal
corr_abs: pd.DataFrame = corr.abs().copy()
np.fill_diagonal(corr_abs.values, 0.0)

# Threshold
threshold: float = 0.95
high_pairs = [
    (col1, col2, corr.loc[col1, col2]) for i, col1 in enumerate(corr.columns)
    for j, col2 in enumerate(corr.columns)
    if j > i and isinstance(corr.loc[col1, col2], (
        float, np.floating)) and abs(corr.loc[col1, col2]) > threshold
]

# 2. Prepare dropdown options
pair_labels: list[str] = [
    f"{a} - {b} (corr={c:.2f})" for a, b, c in high_pairs
]
pair_dict: dict[str, tuple[str, str]] = {
    label: (a, b)
    for label, (a, b, _) in zip(pair_labels, high_pairs)
}
dropdown = Dropdown(options=pair_labels, description="Pair:")

# 3. Global state
index: int = 0
window_size: int = 150
out = Output()


# 4. Plot logic
def plot_selected_pair(start: int, feature1: str, feature2: str) -> None:
    global out
    series1: pd.Series = data[feature1].reset_index(drop=True)
    series2: pd.Series = data[feature2].reset_index(drop=True)
    mean1, std1 = series1.mean(), series1.std()
    mean2, std2 = series2.mean(), series2.std()

    end: int = min(start + window_size, len(series1))
    x = np.arange(start, end)
    y1 = series1[start:end]
    y2 = series2[start:end]

    with out:
        out.clear_output(wait=True)
        plt.figure(figsize=(12, 5))
        plt.plot(x, y1, label=feature1, color="blue")
        plt.plot(x, y2, label=feature2, color="red")

        plt.axhline(mean1, color="blue", linestyle="--")
        plt.axhline(mean1 + std1, color="blue", linestyle=":")
        plt.axhline(mean1 - std1, color="blue", linestyle=":")
        plt.axhline(mean2, color="red", linestyle="--")
        plt.axhline(mean2 + std2, color="red", linestyle=":")
        plt.axhline(mean2 - std2, color="red", linestyle=":")

        plt.title(f"{feature1} vs {feature2} [{start}:{end}]")
        plt.xlabel("Time Index")
        plt.ylabel("Value")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()


# 5. Button callbacks
def on_next_clicked(_):
    global index
    if index + window_size < len(data):
        index += window_size
        f1, f2 = pair_dict[dropdown.value]
        plot_selected_pair(index, f1, f2)


def on_back_clicked(_):
    global index
    if index - window_size >= 0:
        index -= window_size
        f1, f2 = pair_dict[dropdown.value]
        plot_selected_pair(index, f1, f2)


def on_dropdown_change(change):
    global index
    if change["type"] == "change" and change["name"] == "value":
        index = 0  # reset on pair change
        f1, f2 = pair_dict[change["new"]]
        plot_selected_pair(index, f1, f2)


# Bind events
btn_next = Button(description="Next →")
btn_back = Button(description="← Back")
btn_next.on_click(on_next_clicked)
btn_back.on_click(on_back_clicked)
dropdown.observe(on_dropdown_change)

# Init plot
initial_f1, initial_f2 = pair_dict[dropdown.value]
plot_selected_pair(index, initial_f1, initial_f2)

# Show all widgets
VBox([dropdown, HBox([btn_back, btn_next]), out])


In [ ]:
# print the correlation where the upper triangle is greater than 0.90
# This will help identify highly correlated features

corr_matrix = data.corr()

upper_tri = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [
    column for column in upper_tri.columns if any(upper_tri[column] > 1.0)
]
print(len(to_drop))
print(to_drop)

# Fault Classification


## Methode that works with LabelEncoded target variable Y

In [ ]:
# Fit models to the data
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

# Create an instance of each algorithm
# logreg = LogisticRegression(max_iter=10000)
# knn = KNeighborsClassifier()
# dt = DecisionTreeClassifier()
# nb = GaussianNB()
# svm = SVC()

rf = RandomForestClassifier()
xg = xgb.XGBClassifier()

# Train the algorithms on the data
# logreg.fit(x_train, Y_TRAIN)
# knn.fit(x_train, Y_TRAIN)
# dt.fit(x_train, Y_TRAIN)
# nb.fit(x_train, Y_TRAIN)
# svm.fit(x_train, Y_TRAIN)

rf.fit(X_TRAIN, Y_TRAIN_DF)
xg.fit(X_TRAIN, Y_TRAIN_DF)

# Use the trained models to make predictions on new data
# y_pred_logreg =logreg.predict(x_test)
# y_pred_dt = dt.predict(x_test)
# y_pred_nb = nb.predict(x_test)
# y_pred_knn = knn.predict(x_test)
# y_pred_svm = svm.predict(x_test)

y_pred_rf = rf.predict(X_TEST_REDUCED)
y_pred_xg = xg.predict(X_TEST_REDUCED)


In [ ]:
# import joblib
# joblib.dump(logreg, 'exported-models/logistic_regression_model.pkl')
# joblib.dump(dt, 'exported-models/decision_tree_model.pkl')
# joblib.dump(rf, 'exported-models/random_forest_model.pkl')
# joblib.dump(nb, 'exported-models/naive_bayes_model.pkl')
# joblib.dump(knn, 'exported-models/knn_model.pkl')
# joblib.dump(xg, 'exported-models/xgboost_model.pkl')

### From above analysis, XGB and Random forest Performs well.
### Metrics:

In this series, we will use accuracy as the primary metric for evaluating the performance of the different machine learning algorithms. We will update the table as we evaluate the performance of other algorithms in the subsequently.


Average accuracy score obtained for each method, excluding fault No. 9 and 15 (**No feature were Dropped, all 52 sensor measurements were used**)

| Method                                    |Accuracy  |
|-----------------------------------------  |----------|
| XG Boost                                  |  0.887   |
| Neural Network                            |  0.943   |
| 1D CNN-Timeserise                         |  0.892   |
| LSTM-Timeserise                           |  0.924   |
| ANN+RandomForest                          |  0.936   |

## Neural Network

In [ ]:
# DNN Model
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping

# Define input layer
inputs = Input(shape=(X_TRAIN.shape[1], ))

# Define hidden layer with 16 nodes and ReLU activation function
hidden_layer = Dense(100, activation="selu")(inputs)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
# Define output layer with softmax activation function for multi-class classification
outputs = Dense(Y_ENC_TRAIN.shape[1],
                activation="softmax")(hidden_layer)

# Define the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model with binary cross-entropy loss function and Adam optimizer
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

# Print the summary of the model
model.summary()

# Define early stopping callback to monitor validation loss and stop if it doesn't improve for 5 epochs
early_stop = EarlyStopping(monitor="val_loss", patience=5)

# Train the model with 20 epochs and batch size of 32, using the early stopping callback
history = model.fit(
    X_TRAIN,
    Y_ENC_TRAIN,
    epochs=200,
    batch_size=256,
    validation_data=(X_TEST_REDUCED, Y_ENC_TEST_REDUCED),
    callbacks=[early_stop],
)

# Plot the training history for loss and accuracy
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Value")
plt.legend()
plt.show()

In [ ]:
nn_results = model.predict(X_TEST_REDUCED, verbose=0)
y_pred_nn = encoder_1.inverse_transform(nn_results)
y_true_nn = encoder_1.inverse_transform(Y_ENC_TEST_REDUCED)
y_score_nn = model.predict(X_TEST_REDUCED, verbose=0)

In [ ]:
nn_results.shape

## Evaluating

In [ ]:
# Evaluate models and compute metrics
from typing import Union, List, Dict
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
confusion_matrix
)

models_results_list: list[dict] = []
arl_tables_dict: dict[str, pd.DataFrame] = {}
fdr_far_dict: dict[str, pd.DataFrame] = {}
delay_tables_dict: dict[str, pd.DataFrame] = {}
classification_scores_per_fault_tables_dict: dict[str, pd.DataFrame] = {}
fpr_per_class_tables_dict: dict[str, pd.DataFrame] = {}
positive_alarm_rate_per_class_table_dict: dict[str, pd.DataFrame] = {}


def segment_faults(y_true: List[int]) -> List[tuple[int, int, int]]:
    segments = []
    n = len(y_true)
    start = None
    current_fault = 0

    for i in range(n):
        if y_true[i] != 0:
            if current_fault != y_true[i]:
                if current_fault != 0:
                    segments.append((current_fault, start, i - 1))
                current_fault = y_true[i]
                start = i
        else:
            if current_fault != 0:
                segments.append((current_fault, start, i - 1))
                current_fault = 0
                start = None
    if current_fault != 0:
        segments.append((current_fault, start, n - 1))
    return segments


def compute_arl_df(y_true: List[int], y_pred: List[int]) -> pd.DataFrame:
    fault_segments = segment_faults(y_true)
    delays_per_fault: dict[int, List[int]] = {}

    for fault_label, start, end in fault_segments:
        detection_times = [
            t for t in range(start, end + 1) if y_pred[t] == fault_label
        ]
        if detection_times:
            delay = detection_times[0] - start
        else:
            delay = end - start + 1  # penalty for undetected fault
        delays_per_fault.setdefault(fault_label, []).append(delay)

    # ARL1 per fault (mean delay)
    arl1_per_fault = {
        fault: np.mean(delays)
        for fault, delays in delays_per_fault.items()
    }

    # ARL1 weighted overall (all delays equally weighted)
    all_delays = [
        delay for delays in delays_per_fault.values() for delay in delays
    ]
    weighted_overall_arl1 = float(
        np.mean(all_delays)) if all_delays else np.nan

    # ARL1 unweighted overall (mean of per-fault averages)
    unweighted_overall_arl1 = float(np.mean(list(
        arl1_per_fault.values()))) if arl1_per_fault else np.nan

    # Compute ARL0 once for all faults (false alarm interval)
    normal_indices = [i for i, v in enumerate(y_true) if v == 0]
    false_alarm_times = [i for i in normal_indices if y_pred[i] != 0]

    if len(false_alarm_times) < 2:
        arl0 = float(
            'inf') if len(false_alarm_times
                          ) == 0 else normal_indices[-1] - false_alarm_times[0]
    else:
        intervals = [
            j - i
            for i, j in zip(false_alarm_times[:-1], false_alarm_times[1:])
        ]
        arl0 = float(np.mean(intervals)) if intervals else float('inf')

    # Create DataFrame
    df = pd.DataFrame({
        'Fault': list(arl1_per_fault.keys()),
        'ARL1': list(arl1_per_fault.values()),
        'ARL0': [arl0] * len(arl1_per_fault)
    }).set_index('Fault')

    # Append average row with both overall ARL1 versions and ARL0
    avg_row = pd.DataFrame({
        'ARL1': [unweighted_overall_arl1],
        'ARL0': [arl0]
    },
                           index=['Average (unweighted)'])

    weighted_row = pd.DataFrame(
        {
            'ARL1': [weighted_overall_arl1],
            'ARL0': [arl0]
        },
        index=['Average (weighted)'])

    df = pd.concat([df, avg_row, weighted_row])

    return df


def compute_fdr_far(
    y_true: List[int],
    y_pred: List[int]
) -> pd.DataFrame:
    y_true_arr = np.array(y_true)
    y_pred_arr = np.array(y_pred)
    classes = sorted(set(y_true_arr) | set(y_pred_arr))
    classes = [c for c in classes if c != 0]  # exclude class 0 (normal)

    results: Dict[str, List[float]] = {"Class": [], "FDR": [], "FAR": [], "Support": []}

    for c in classes:
        tp = np.sum((y_true_arr == c) & (y_pred_arr == c))
        fp = np.sum((y_true_arr != c) & (y_pred_arr == c))
        fn = np.sum((y_true_arr == c) & (y_pred_arr != c))

        support = tp + fn  # number of true instances of this class

        fdr = fp / (tp + fp) if (tp + fp) > 0 else np.nan
        far = fp / np.sum(y_pred_arr != 0) if np.sum(y_pred_arr != 0) > 0 else np.nan

        results["Class"].append(c)
        results["FDR"].append(fdr)
        results["FAR"].append(far)
        results["Support"].append(support)

    df = pd.DataFrame(results).set_index("Class")

    # Unweighted mean
    avg_row = pd.DataFrame({
        "FDR": [df["FDR"].mean()],
        "FAR": [df["FAR"].mean()],
        "Support": [df["Support"].sum()]
    }, index=["Average (unweighted)"])

    # Weighted mean
    weights = df["Support"] / df["Support"].sum()
    weighted_row = pd.DataFrame({
        "FDR": [(df["FDR"] * weights).sum()],
        "FAR": [(df["FAR"] * weights).sum()],
        "Support": [df["Support"].sum()]
    }, index=["Average (weighted)"])

    df = pd.concat([df, avg_row, weighted_row])
    return df


def combine_arl_fdr_far(df_arl: pd.DataFrame,
                        df_fdr_far: pd.DataFrame) -> pd.DataFrame:
    # Convert index to string for safe filtering
    df_arl.index = df_arl.index.map(str)
    df_fdr_far.index = df_fdr_far.index.map(str)

    # Filter out "Average" rows from both
    df_arl_main = df_arl[~df_arl.index.str.contains("Average")]
    df_fdr_far_main = df_fdr_far[~df_fdr_far.index.str.contains("Average")]

    # Join on class index
    df_combined = df_arl_main.join(df_fdr_far_main, how="outer")

    # Add average rows (now strings)
    avg_rows = pd.concat([
        df_arl[df_arl.index.str.contains("Average")],
        df_fdr_far[df_fdr_far.index.str.contains("Average")]
    ],
                         axis=0)

    return pd.concat([df_combined, avg_rows])

def compute_classification_scores_per_fault(
        y_true: Union[List[int], np.ndarray],
        y_pred: Union[List[int], np.ndarray]) -> pd.DataFrame:
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    fault_types = np.unique(y_true[y_true > 0])
    scores_per_fault = {}

    for fault in fault_types:
        y_true_binary = (y_true == fault).astype(int)
        y_pred_binary = (y_pred == fault).astype(int)

        accuracy = accuracy_score(y_true_binary, y_pred_binary)
        precision = precision_score(y_true_binary,
                                    y_pred_binary,
                                    zero_division=0)
        recall = recall_score(y_true_binary, y_pred_binary, zero_division=0)
        f1 = f1_score(y_true_binary, y_pred_binary, zero_division=0)

        scores_per_fault[fault] = {
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-score": f1,
        }

    return pd.DataFrame(scores_per_fault)

def macro_false_alarm_rate(y_true: Union[list[int], np.ndarray],
                           y_pred: Union[list[int], np.ndarray]) -> float:
    # Compute the confusion matrix for multi-class classification
    cm: np.ndarray = confusion_matrix(y_true, y_pred)

    # Get the number of classes from the confusion matrix shape
    n_classes: int = cm.shape[0]

    # List to store false positive rate (FPR) for each class
    fpr_list: list[float] = []

    # Iterate over each class index (one-vs-rest treatment)
    for i in range(n_classes):
        # False positives for class i: sum of predictions as class i
        # excluding the true positives on the diagonal
        fp: int = cm[:, i].sum() - cm[i, i]

        # True negatives: all samples excluding the positives for class i
        # and the false positives for class i
        tn: int = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])

        # Compute FPR: FP / (FP + TN), guard against division by zero
        fpr: float = fp / (fp + tn) if (fp + tn) > 0 else 0.0

        # Append this class’s FPR to the list
        fpr_list.append(fpr)

    # Return the macro-average of FPRs across all classes
    return float(np.mean(fpr_list).item())

def false_alarm_rate_per_class(
        y_true: Union[list[int], np.ndarray],
        y_pred: Union[list[int], np.ndarray]) -> pd.DataFrame:
    # Compute confusion matrix
    cm: np.ndarray = confusion_matrix(y_true, y_pred)

    # Number of classes
    n_classes: int = cm.shape[0]

    # Dictionary to store FPR per class
    fpr_per_class: dict[int, float] = {}

    # Compute FPR for each class using one-vs-rest
    for i in range(n_classes):
        # False positives for class i
        fp: int = cm[:, i].sum() - cm[i, i]

        # True negatives for class i
        tn: int = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])

        # FPR: FP / (FP + TN), avoid division by zero
        fpr: float = fp / (fp + tn) if (fp + tn) > 0 else 0.0

        # Store in dictionary
        fpr_per_class[i] = fpr

    return pd.DataFrame.from_dict(fpr_per_class,
                                  orient="index",
                                  columns=["FPR"])

def positive_alarm_rate_per_class(
        y_true: Union[list[int], np.ndarray],
        y_pred: Union[list[int], np.ndarray]) -> pd.DataFrame:
    cm: np.ndarray = confusion_matrix(y_true, y_pred)
    n_classes: int = cm.shape[0]
    par_per_class: dict[int, float] = {}

    for i in range(n_classes):
        tp: int = cm[i, i]
        fn: int = cm[i, :].sum() - tp
        par: float = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        par_per_class[i] = par

    return pd.DataFrame.from_dict(par_per_class,
                                  orient="index",
                                  columns=["PAR"])

def evaluate_model(model_name: str, y_true, y_pred) -> None:
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true,y_pred,average="macro",zero_division=0)
    rec = recall_score(y_true, y_pred, average="macro", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
    far = macro_false_alarm_rate(y_true, y_pred)
    models_results_list.append({
        "Model": model_name,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1-Score": f1,
        "False Alarm Rate": far,  # or False Positive Rate (FPR)
    })
    print(" ")
    print("Model name: ", model_name)
    print("compute arl fdr far ")
    arl_tables_dict[model_name] = compute_arl_df(y_true, y_pred)
    fdr_far_dict[model_name] = compute_fdr_far(y_true, y_pred)
    delay_tables_dict[model_name] = compute_first_detection_delay(y_true, y_pred)
    print("compute_first_detection_delay ")
    classification_scores_per_fault_tables_dict[model_name] = (compute_classification_scores_per_fault(y_true, y_pred))
    print("classification_scores_per_fault_tables ")
    fpr_per_class_tables_dict[model_name] = false_alarm_rate_per_class(y_true, y_pred)
    print("false_alarm_rate_per_class ")
    positive_alarm_rate_per_class_table_dict[model_name] = positive_alarm_rate_per_class( y_true, y_pred)
    print("positive_alarm_rate_per_class ")


# Evaluate
evaluate_model("Random Forest", Y_TEST_REDUCED, y_pred_rf)
evaluate_model("XGBoost", Y_TEST_REDUCED, y_pred_xg)
evaluate_model("Neural Net", Y_TEST_REDUCED, y_pred_nn.ravel())


### Table average metrics

In [ ]:
model_classification_average=pd.DataFrame(models_results_list)
save_dataframe(df=model_classification_average, name="Model Results average comparing")
model_classification_average.head()

### Plot average metrics 

In [ ]:
# plot_models_metrics_compareing
def plot_models_metrics_compareing(models_results,
                                   plot_name: str = "metric_average") -> None:
    results_df = pd.DataFrame(models_results)
    # metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Detection Rate', 'Avg Delay', 'ARL₁', 'ARL₀', 'False Alarm Rate']
    metrics = [
        "Accuracy",
        "Precision",
        "Recall",
        "F1-Score",
        #"False Alarm Rate",
    ]
    melted_df = results_df.melt(
        id_vars="Model",
        value_vars=metrics,
        var_name="Metric",
        value_name="Score",
    )

    plt.figure(figsize=(16, 6))
    ax = sns.barplot(data=melted_df, x="Metric", y="Score", hue="Model")
    ax.grid(axis="y", linestyle="--", linewidth=0.9, alpha=0.9)
    plt.title("Model Comparison Across Metrics")
    plt.ylim(0, 1.0)
    plt.legend(title="Model", bbox_to_anchor=(1.01, 1), loc="upper left")
    plt.tight_layout()
    save_plot(plot_name=plot_name, plot_path="average")
    plt.show()


plot_models_metrics_compareing(models_results_list)

### Table extended metrics average

In [ ]:
# Compute extended metrics average

from os import name
from typing import Union
import numpy as np
import pandas as pd
from tabulate import tabulate
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd





def compute_average_classification_metrics(
        y_true: Union[list[int], np.ndarray],
        y_pred: Union[list[int], np.ndarray]) -> pd.DataFrame:
    cm: np.ndarray = confusion_matrix(y_true, y_pred)
    n_classes: int = cm.shape[0]

    # Initialize accumulators
    acc_list: list[float] = []
    prec_list: list[float] = []
    rec_list: list[float] = []
    tnr_list: list[float] = []
    fpr_list: list[float] = []
    fnr_list: list[float] = []
    npv_list: list[float] = []
    fdr_list: list[float] = []
    bal_acc_list: list[float] = []
    f1_list: list[float] = []

    for i in range(n_classes):
        tp: int = cm[i, i]
        fn: int = cm[i, :].sum() - tp
        fp: int = cm[:, i].sum() - tp
        tn: int = cm.sum() - (tp + fp + fn)

        # Add safe guards to avoid division by zero
        acc: float = ((tp + tn) / (tp + tn + fp + fn) if
                      (tp + tn + fp + fn) > 0 else 0.0)
        prec: float = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        rec: float = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        tnr: float = tn / (tn + fp) if (tn + fp) > 0 else 0.0
        fpr: float = fp / (fp + tn) if (fp + tn) > 0 else 0.0
        fnr: float = fn / (tp + fn) if (tp + fn) > 0 else 0.0
        npv: float = tn / (tn + fn) if (tn + fn) > 0 else 0.0
        fdr: float = fp / (tp + fp) if (tp + fp) > 0 else 0.0
        bal_acc: float = (rec + tnr) / 2
        f1: float = (2 * prec * rec) / (prec + rec) if (prec +
                                                        rec) > 0 else 0.0

        # Accumulate
        acc_list.append(acc)
        prec_list.append(prec)
        rec_list.append(rec)
        tnr_list.append(tnr)
        fpr_list.append(fpr)
        fnr_list.append(fnr)
        npv_list.append(npv)
        fdr_list.append(fdr)
        bal_acc_list.append(bal_acc)
        f1_list.append(f1)

    # Macro-average
    metrics: dict[str, float] = {
        "Macro accuracy": float(np.mean(acc_list)),  # "How often am I correct overall, regardless of class?"
        "Precision": float(np.mean(prec_list)),  # "When I flag something, how often is it truly an issue?"
        "Recall / TPR": float(np.mean(rec_list)),  # "Out of all actual issues, how many did I correctly flag?"
        "F1-Score": float(np.mean(f1_list)),  # "What’s the balance between being thorough and being right when flagging?"
        "FPR": float(np.mean(fpr_list)),  # "How often do I wrongly flag a normal case?"
        "NPV (Negative Predictive Value)": float(np.mean(npv_list)),  # "When I say something is normal, how often is that actually true?"
        "Balanced Accuracy": float(np.mean(bal_acc_list)),  # "How well do I perform on both classes, accounting for imbalance?"
        # "FDR (False Discovery Rate)": float(np.mean(fdr_list)),  # "When I raise an alert, how often am I wrong?" FDR = 1 − Precision
        # "FNR (False Negative Rate)": float(np.mean(fnr_list)),  # "How often do I miss a real issue?" FNR = 1 − Recall
        # "TNR": float(np.mean(tnr_list)),  # "Out of all normal cases, how many did I correctly leave alone?" TNR = 1 − FPR
    }


    return pd.DataFrame.from_dict(metrics, orient="index", columns=["Value"])


# Compute metrics individually
metrics_xg: pd.DataFrame = compute_average_classification_metrics(Y_TEST_REDUCED, y_pred_xg)
metrics_rf: pd.DataFrame = compute_average_classification_metrics(Y_TEST_REDUCED, y_pred_rf)
metrics_nn: pd.DataFrame = compute_average_classification_metrics(Y_TEST_REDUCED, y_pred_nn)

# Rename the columns to indicate the model
metrics_xg.columns = [XGBOOST]
metrics_rf.columns = [RANDOM_FOREST]
metrics_nn.columns = [NEURAL_NET]

# Concatenate into a single table (column-wise)
concated_average_scores_metrics: pd.DataFrame = pd.concat([metrics_xg, metrics_rf, metrics_nn], axis=1)

# Print formatted table
print(tabulate(concated_average_scores_metrics, headers="keys", tablefmt="grid", showindex=True))
save_dataframe(df=concated_average_scores_metrics,name="score average")


### Plots metrics score average

In [ ]:
def plot_score_metrics(metrics_df: pd.DataFrame) -> None:
    """
    Plots a grouped bar chart of classification metrics.

    Args:
        metrics_df (pd.DataFrame): DataFrame with metrics as rows and model names as columns.
    """
    # Reset index to get metric names as a column
    df: pd.DataFrame = metrics_df.reset_index().melt(id_vars="index",
                                                     var_name="Model",
                                                     value_name="Value")
    df.rename(columns={"index": "Metric"}, inplace=True)

    plt.figure(figsize=(12, 6))
    sns.barplot(data=df, x="Metric", y="Value", hue="Model")

    plt.title("Classification Metrics by Model")
    plt.xticks(rotation=45, ha="right")
    plt.yticks(np.linspace(0, 1, 21))  # Adds grid lines at intervals of 0.05
    plt.yticks(fontsize=8)
    plt.grid(axis="y", linestyle="--", linewidth=0.5)
    plt.ylim(0, 1.05)
    plt.legend(title="Model")
    plt.tight_layout()
    save_plot(plot_name="score average", plot_path="average")
    plt.show()

plot_score_metrics(concated_average_scores_metrics)

### Tables per fault metric comparing

In [ ]:
# Compute & Print classification scores tables per fault for each model
from typing import Union
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix


def compute_classification_metrics_per_class(
        y_true: Union[list[int], np.ndarray],
        y_pred: Union[list[int], np.ndarray]) -> pd.DataFrame:
    cm: np.ndarray = confusion_matrix(y_true, y_pred)
    n_classes: int = cm.shape[0]
    metrics: list[dict] = []

    for i in range(n_classes):
        tp: int = cm[i, i]
        fn: int = cm[i, :].sum() - tp
        fp: int = cm[:, i].sum() - tp
        tn: int = cm.sum() - (tp + fp + fn)

        acc: float = ((tp + tn) / (tp + tn + fp + fn) if
                      (tp + tn + fp + fn) > 0 else 0.0)
        prec: float = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        rec: float = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        tnr: float = tn / (tn + fp) if (tn + fp) > 0 else 0.0
        fpr: float = fp / (fp + tn) if (fp + tn) > 0 else 0.0
        fnr: float = fn / (tp + fn) if (tp + fn) > 0 else 0.0
        npv: float = tn / (tn + fn) if (tn + fn) > 0 else 0.0
        fdr: float = fp / (tp + fp) if (tp + fp) > 0 else 0.0
        bal_acc: float = (rec + tnr) / 2
        f1: float = (2 * prec * rec) / (prec + rec) if (prec +
                                                        rec) > 0 else 0.0

        metrics.append({
            "Class": i,
            "Accuracy": acc,  # "How often am I correct overall, regardless of class?"
            "Precision": prec,  # "When I flag something, how often is it truly an issue?"
            "Recall / TPR": rec,  # "Out of all actual issues, how many did I correctly flag?"
            "F1-Score": f1,  # "What’s the balance between being thorough and being right when flagging?"
            "FPR": fpr,  # "How often do I wrongly flag a normal case?"
            "NPV (Negative Predictive Value)": npv,  # "When I say something is normal, how often is that actually true?"
            "Balanced Accuracy": bal_acc,  # "How well do I perform on both classes, accounting for imbalance?"
            # "FDR (False Discovery Rate)": fdr,  # "When I raise an alert, how often am I wrong?" FDR = 1 − Precision
            # "FNR (False Negative Rate)": fnr,  # "How often do I miss a real issue?" FNR = 1 − Recall
            # "TNR": tnr,  # "Out of all normal cases, how many did I correctly leave alone?" TNR = 1 − FPR
        })


    return pd.DataFrame(metrics).set_index("Class")


classification_results_per_model: dict[str, pd.DataFrame] = {
    "Random Forest":
    compute_classification_metrics_per_class(Y_TEST_REDUCED, y_pred_rf),
    "XGBoost":
    compute_classification_metrics_per_class(Y_TEST_REDUCED, y_pred_xg),
    "Neural Net":
    compute_classification_metrics_per_class(Y_TEST_REDUCED, y_pred_nn),
}

# loop through each model and print the classification metrics
for model_name, df in classification_results_per_model.items():
    # Add an "Average" row at the end of the DataFrame
    average_row = df.mean(axis=0).to_frame().T
    average_row.index = ["Average"]
    df = pd.concat([df, average_row])
    save_dataframe(df=df, name=model_name, suffix="metrics")
    print(f"Classification Metrics for {model_name}:")
    print(tabulate(df, headers="keys", tablefmt="grid"))
    print("\n")

### Plots metrics per Fault

In [ ]:
# Plot Compute per-class metrics for each comparison
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np




def plot_per_class_metrics_comparison(
        model_metrics: dict[str, pd.DataFrame],
        plot_name: str = "metric_fault_number") -> None:
    model_names: list[str] = list(model_metrics.keys())
    classes: list[int] = model_metrics[model_names[0]].index.tolist()
    metrics: list[str] = model_metrics[model_names[0]].columns.tolist()

    for class_id in classes:
        # Transpose to shape [metric x model]
        plot_df = pd.DataFrame({
            model: df.loc[class_id] for model, df in model_metrics.items()
        })
        plot_df = plot_df.T  # [models x metrics]

        # Transpose to group by metric
        plot_df = plot_df.T  # [metrics x models]

        ax = plot_df.plot(kind="bar", figsize=(20, 8), width=0.75)
        ax.set_title(f"Fault Class {class_id} – Model Comparison per Metric")
        ax.set_xlabel("Metric")
        ax.set_ylabel("Score")
        ax.set_ylim(0, 1)
        ax.set_xticks(np.arange(len(metrics)))
        ax.set_xticklabels(metrics, rotation=45, ha="right")

        # Add fine horizontal grid lines every 0.05
        ax.yaxis.set_ticks(np.arange(0, 1.05, 0.05))
        ax.grid(axis="y", linestyle="--", alpha=0.5)

        ax.legend(title="Model", bbox_to_anchor=(1.05, 1), loc="upper left")
        plt.tight_layout()
        save_plot(plot_name=plot_name, suffix=str(class_id), plot_path="per_fault")
        plt.show()

    # Average per model
    avg_df = pd.DataFrame({
        model: df.mean(axis=0)
        for model, df in model_metrics.items()
    })
    avg_df = avg_df.T  # [models x metrics]
    avg_df = avg_df.T  # [metrics x models]

    ax = avg_df.plot(kind="bar", figsize=(20, 8), width=0.75)
    ax.set_title("Average Metrics Across All Classes – Model Comparison")
    ax.set_xlabel("Metric")
    ax.set_ylabel("Average Score")
    ax.set_ylim(0, 1)
    ax.set_xticks(np.arange(len(metrics)))
    ax.set_xticklabels(metrics, rotation=45, ha="right")

    ax.yaxis.set_ticks(np.arange(0, 1.05, 0.1))
    ax.grid(axis="y", linestyle="--", alpha=0.5)

    ax.legend(title="Model", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    save_plot(plot_name=plot_name, suffix="average",plot_path="average")
    plt.show()


plot_per_class_metrics_comparison(classification_results_per_model)


### Plots per fault per Metric

In [ ]:
# Plotting classification scores for all models
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


def plot_all_fault_scores_per_metric_all_models(
        classification_scores_per_fault_tables: dict[str, dict],
        plot_name: str = "Fault_per_Metric") -> None:
    # Flatten input dict into list of records
    records = []
    for (
            model_name,
            score_dict,
    ) in classification_scores_per_fault_tables.items():
        df = score_dict
        for metric in df.index:
            for fault in df.columns:
                records.append({
                    "Model": model_name,
                    "Fault": int(fault),  # ensure numerical sorting
                    "Metric": metric,
                    "Score": df.loc[metric, fault],
                })

    combined_df = pd.DataFrame(records)
    metrics = combined_df["Metric"].unique()
    faults = sorted(combined_df["Fault"].unique())
    models = combined_df["Model"].unique()

    # Plot one subplot per metric
    for metric in metrics:
        metric_df = combined_df[combined_df["Metric"] == metric]

        # Pivot: rows=faults, columns=models, values=scores
        pivot_df = metric_df.pivot(index="Fault",
                                   columns="Model",
                                   values="Score").loc[faults]

        group_spacing: float = 3.0  # control the group spacing
        x: np.ndarray = np.arange(0, len(faults) * group_spacing, group_spacing)
        bar_width: float = group_spacing * 0.8 / len(models)

        plt.figure(figsize=(20, 4))
        for i, model in enumerate(models):
            plt.bar(x + i * bar_width,
                    pivot_df[model],
                    width=bar_width,
                    label=model)

        plt.title(f"{metric}")
        plt.xlabel("Fault")
        plt.ylabel("Score")
        plt.xticks(x + bar_width * (len(models) - 1) / 2, faults)
        plt.ylim(0, 1)
        plt.yticks(np.arange(0, 1.05, 0.05))
        plt.grid(axis="y", linestyle="--", alpha=0.5)
        plt.legend(title="Model", bbox_to_anchor=(1.05, 1), loc="upper left")
        plt.tight_layout()
        save_plot(plot_name=plot_name, suffix=str(metric),plot_path="per_metric")
        plt.show()


plot_all_fault_scores_per_metric_all_models(classification_scores_per_fault_tables_dict)

### Plots Confusion Matrices

In [ ]:
# Plot Confusion Matrices
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix

from tabulate import tabulate
from typing import Union, List


def plot_confusion_matrix(
    y_true: Union[List[int], np.ndarray],
    y_pred: Union[List[int], np.ndarray],
    title: str = "confusion_matrix",
) -> None:
    cm = confusion_matrix(y_true, y_pred, normalize="true")
    num_classes: int = cm.shape[0]

    width: float = max(6, num_classes * 0.8)
    height: float = max(4, num_classes * 0.6)

    f, ax = plt.subplots(figsize=(width, height))
    sns.heatmap(cm, annot=True, cmap="Blues", ax=ax)
    ax.set_title(title)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("Actual")
    ax.set_ylim(num_classes, 0)
    plt.tight_layout()
    save_plot(plot_name=title,
              suffix="confusion_matrix",
              plot_path="confusion_matrix")
    plt.show()


def plot_all_confusion_matrices(model_preds: dict[str, tuple]) -> None:
    for name, (y_true, y_pred) in model_preds.items():
        plot_confusion_matrix(y_true, y_pred, f"{name} Confusion Matrix")


plot_all_confusion_matrices({
    "Random Forest": (Y_TEST_REDUCED, y_pred_rf),
    "XGBoost": (Y_TEST_REDUCED, y_pred_xg),
    "Neural Net": (y_true_nn, y_pred_nn),
})

### Tables for ARL per model

### Plots ARL

**ARL**
- Low ARL₀ is bad for practical use: causes many unnecessary alarms.

- Low ARL₁ is good if it's accompanied by high ARL₀ — i.e., a sharp and correct detection only when needed.

- Low ARL₀ and ARL₁ mean the feature triggers many false alarms and detects faults too early or too easily. It's likely unstable or noisy, not reliable alone.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


def plot_arl_tables_dict(arl_tables_dict: dict[str, pd.DataFrame]) -> None:
    for model_name, df_arl in arl_tables_dict.items():
        df_arl = df_arl.copy()
        df_arl.index = df_arl.index.map(str)

        # Per-fault (exclude average rows)
        df_faults = df_arl[~df_arl.index.str.contains("Average")]
        df_faults = df_faults[["ARL1", "ARL0"]].dropna(how="all")

        x = range(len(df_faults))
        width = 0.5

        fig, ax = plt.subplots(figsize=(25, 6))
        ax.bar([i - width / 2 for i in x],
               df_faults["ARL1"],
               width,
               label="ARL1",
               color="mediumseagreen")
        ax.bar([i + width / 2 for i in x],
               df_faults["ARL0"],
               width,
               label="ARL0",
               color="coral")

        ax.set_xticks(list(x))
        ax.set_xticklabels(df_faults.index, rotation=45, ha="right")
        ax.set_ylabel("Samples")
        ax.set_title(f"ARL Metrics per Fault - Model: {model_name}")
        ax.legend()
        ax.grid(axis="y", linestyle="--", alpha=0.6)

        for i in x:
            for j, col in enumerate(["ARL1", "ARL0"]):
                val = df_faults.iloc[i][col]
                if pd.notna(val):
                    ax.text(i + (j - 0.5) * width,
                            val + 0.02 * max(df_faults.max()),
                            f"{val:.2f}",
                            ha="center",
                            fontsize=8)

        plt.tight_layout()
        save_plot(plot_name="arl per class", plot_path="arl")
        plt.show()

        # Plot averages separately
        df_avg = df_arl[df_arl.index.str.contains("Average")]
        if not df_avg.empty:
            df_avg = df_avg[["ARL1", "ARL0"]].dropna(how="all")
            df_avg.plot(kind="bar",
                        figsize=(5,6),
                        color=["mediumseagreen", "coral"])
            plt.title(f"ARL Averages - Model: {model_name}")
            plt.ylabel("Samples")
            plt.xticks(rotation=0)
            plt.grid(axis="y", linestyle="--", alpha=0.6)
            ax = plt.gca()
            for container in ax.containers:
                ax.bar_label(container, fmt="%.2f", padding=3)
            plt.tight_layout()
            save_plot(plot_name="arl average", plot_path="arl")
            plt.show()


plot_arl_tables_dict(arl_tables_dict)


### Plot FDR FAR



**🔹 WHAT the Metrics Mean**

**1. ARL₁ (Average Run Length for Faults)**

* Definition: Expected number of time steps **after a fault starts** until it is detected.
* Low ARL₁ = **fast detection**
* High ARL₁ = **slow detection**

**2. FDR (False Detection Rate)**

* Definition: Among all times the model predicted a specific fault (e.g. fault 2), how many were **wrong**.
* High FDR = **many predictions of fault 2 were incorrect** (i.e., mislabeling)
* FDR = FP / (TP + FP) per fault class

**3. FAR (False Alarm Rate)**

* Definition: How often a fault class is **falsely predicted** during **normal operation** (i.e. when the true class is 0).
* Measures how noisy the model is during normal time.
* FAR = FP / (# total predicted faults during normal)

---

**🔹 HOW They Relate**

| Metric                                   | Condition                                                            | Interpretation                        |
| ---------------------------------------- | -------------------------------------------------------------------- | ------------------------------------- |
| **Low ARL₁** + **Low FDR** + **Low FAR** | Ideal                                                                | Fast and accurate detection, no noise |
| **Low ARL₁** + **High FDR**              | Fast detection, but many wrong fault predictions                     |                                       |
| **High ARL₁** + **Low FDR**              | Late detection, but when it happens, it's correct                    |                                       |
| **High FAR**                             | Model predicts faults during normal → false alarms                   |                                       |
| **High FDR** + **High FAR**              | Model often predicts faults, and often wrongly — unstable classifier |                                       |

---

**🔹 HOW to Interpret the Plot**

In your **combined plot**:

* Each bar group (per fault) shows:

  * **ARL₁:** How quickly the fault is detected
  * **FDR:** How often the predicted label for this fault was wrong
  * **FAR:** How noisy this fault label is during normal time

---

**🔹 Example Interpretation**

Say for **Fault 2**:

* ARL₁ = 0.5 → detected fast
* FDR = 0.3 → 30% of “Fault 2” predictions were incorrect
* FAR = 0.2 → Fault 2 was predicted during normal periods 20% of the time

**Interpretation**:

* Detection is quick (good),
* but 30% of the time the system thinks it's Fault 2, it's actually not (bad),
* and it's somewhat noisy during normal operation.

You'd ask:

* Can we improve the model to reduce false positives for Fault 2?
* Is the model too biased toward predicting Fault 2?

---

## 🔹 Summary Table

| Metric | Low is Good? | Meaning                   |
| ------ | ------------ | ------------------------- |
| ARL₁   | ✅            | Faster fault detection    |
| FDR    | ✅            | More precise labeling     |
| FAR    | ✅            | More stable during normal |

---



In [ ]:
def plot_fdr_far_dict(fdr_far_dict: dict[str, pd.DataFrame]) -> None:
    for model_name, df_fdr_far in fdr_far_dict.items():
        df = df_fdr_far.copy()
        df.index = df.index.map(str)

        # Per-fault (exclude averages)
        df_faults = df[~df.index.str.contains("Average")]
        df_faults = df_faults[["FDR", "FAR"]].dropna(how="all")

        x = range(len(df_faults))
        width = 0.5

        fig, ax = plt.subplots(figsize=(25, 6))
        ax.bar([i - width / 2 for i in x],
               df_faults["FDR"],
               width,
               label="FDR",
               color="cornflowerblue")
        ax.bar([i + width / 2 for i in x],
               df_faults["FAR"],
               width,
               label="FAR",
               color="salmon")

        ax.set_xticks(list(x))
        ax.set_xticklabels(df_faults.index, rotation=45, ha="right")
        ax.set_ylabel("Rate")
        ax.set_title(f"FDR and FAR per Fault - Model: {model_name}")
        ax.legend()
        ax.grid(axis="y", linestyle="--", alpha=0.6)

        for i in x:
            for j, col in enumerate(["FDR", "FAR"]):
                val = df_faults.iloc[i][col]
                if pd.notna(val):
                    ax.text(i + (j - 0.5) * width,
                            val + 0.015,
                            f"{val:.2f}",
                            ha="center",
                            fontsize=8)

        plt.tight_layout()
        save_plot(plot_name="fdr_far per class", plot_path="fdr_far")
        plt.show()

        # Plot averages separately
        df_avg = df[df.index.str.contains("Average")]
        if not df_avg.empty:
            df_avg = df_avg[["FDR", "FAR"]].dropna(how="all")
            df_avg = df_avg.rename_axis("Average Type").reset_index()
            df_melted = df_avg.melt(id_vars="Average Type",
                                    var_name="Metric",
                                    value_name="Value")

            plt.figure(figsize=(5, 6))
            ax = sns.barplot(data=df_melted,
                             x="Average Type",
                             y="Value",
                             hue="Metric",
                             palette={
                                 "FDR": "cornflowerblue",
                                 "FAR": "salmon"
                             })

            ax.set_title(f"FDR and FAR Averages - Model: {model_name}")
            ax.set_ylabel("Rate")
            ax.set_xlabel("")
            ax.grid(axis="y", linestyle="--", alpha=0.6)

            for container in ax.containers:
                ax.bar_label(container, fmt="%.2f", padding=3)

            plt.tight_layout()
            save_plot(plot_name="fdr_far average", plot_path="fdr_far")
            plt.show()


plot_fdr_far_dict(fdr_far_dict)


### Plot first correct detection delay

In [ ]:
# Plot_combined_detection_delay
def plot_combined_detection_delay(delay_tables: dict,
                                  plot_name: str = "detection_delay") -> None:
    combined_df = pd.concat(
        [df.assign(Model=model) for model, df in delay_tables.items()],
        ignore_index=True,
    )

    plt.figure(figsize=(14, 6))
    sns.barplot(data=combined_df,
                x="Fault",
                y="First Detection Delay",
                hue="Model")
    plt.title("First Correct Detection Delay per Fault for All Models")
    plt.xlabel("Fault")
    plt.ylabel("First Detection Delay (samples)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    save_plot(plot_name=plot_name, plot_path="detection_delay")
    plt.show()


plot_combined_detection_delay(delay_tables_dict)

# Anomaly Detection
 ** No fault classification**

In [ ]:
# Global Variables
anomaly_results_per_model: dict[str, pd.DataFrame] = {}

## MCUSUM

In [ ]:
# MCUSUM Code

import numpy as np
from numpy.typing import NDArray


def estimate_incontrol_parameters(
    X_incontrol: NDArray[np.float64],
) -> tuple[NDArray[np.float64], NDArray[np.float64]]:
    mu_0 = np.mean(X_incontrol, axis=0)
    sigma = np.cov(X_incontrol, rowvar=False, bias=False)
    return mu_0, sigma


def compute_mcusum_scores(
    X_test: NDArray[np.float64],
    mu_0: NDArray[np.float64],
    sigma: NDArray[np.float64],
    k: float,
) -> NDArray[np.float64]:
    X_test = np.asarray(X_test)
    mu_0 = np.asarray(mu_0)
    sigma = np.asarray(sigma)

    n_samples, n_features = X_test.shape

    eigvals, eigvecs = np.linalg.eigh(sigma)
    eigvals_inv_sqrt = np.diag(1.0 / np.sqrt(eigvals))
    sigma_inv_sqrt = eigvecs @ eigvals_inv_sqrt @ eigvecs.T

    Z = (X_test - mu_0) @ sigma_inv_sqrt.T

    S_t = np.zeros(n_features)
    T = np.zeros(n_samples)

    for t in range(n_samples):
        V_t = S_t + Z[t]
        norm_V_t = np.linalg.norm(V_t)

        if norm_V_t <= k:
            S_t = np.zeros(n_features)
        else:
            shrinkage = 1.0 - k / norm_V_t
            S_t = V_t * shrinkage

        T[t] = np.linalg.norm(S_t)

    return T


def compute_reference_value_k(delta: NDArray[np.float64],
                              sigma: NDArray[np.float64]) -> float:
    """
    Compute MCUSUM reference value k = 0.5 * ||Σ^{-1/2} δ||

    Parameters:
    - delta: shift vector (length = n_features)
    - sigma: in-control covariance matrix (n_features x n_features)

    Returns:
    - k: reference value
    """

    # Whitening matrix: Σ^{-1/2} using eigen decomposition
    eigvals, eigvecs = np.linalg.eigh(sigma)
    eigvals_inv_sqrt = np.diag(1.0 / np.sqrt(eigvals))
    sigma_inv_sqrt = eigvecs @ eigvals_inv_sqrt @ eigvecs.T

    # Transform delta into whitened space
    whitened_delta = sigma_inv_sqrt @ delta

    # Compute norm and halve it
    k = 0.5 * np.linalg.norm(whitened_delta)

    return k


def estimate_h(
    x_incontrol_np: NDArray[np.float64],
    k: float = 0.5,
    percential_threshold: int = 98,
) -> float:
    """
    Estimate in-control parameters mu_0 and sigma from in-control data.
    """
    mu_0, sigma = estimate_incontrol_parameters(x_incontrol_np)

    n_simulations = 500
    max_T_values = []

    for i in range(n_simulations):
        indices = np.random.choice(x_incontrol_np.shape[0],
                                   size=300,
                                   replace=True)
        sample = x_incontrol_np[indices]
        T = compute_mcusum_scores(sample, mu_0, sigma, k=k)
        max_T_values.append(np.max(T))

    # Empirical percential_threshold for false alarm
    h = np.percentile(max_T_values, percential_threshold)
    print("Estimated control limit h:", h)
    return h


def get_control_limit(alpha: float, n_features: int) -> float:
    return chi2.ppf(1 - alpha, df=n_features)


def compute_detection_metrics(
        predicted_flags: NDArray[np.bool_],
        true_labels: NDArray[np.int64]) -> dict[str, float | str]:
    y_pred = predicted_flags.astype(int)
    y_true = true_labels.astype(int)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc: float = (tp + tn) / (tp + tn + fp + fn)
    prec: float = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    rec: float = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    tnr: float = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    fpr: float = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    fnr: float = fn / (fn + tp) if (fn + tp) > 0 else 0.0
    npv: float = tn / (tn + fn) if (tn + fn) > 0 else 0.0
    fdr: float = fp / (fp + tp) if (fp + tp) > 0 else 0.0
    bal_acc: float = 0.5 * (rec + tnr)
    f1: float = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0.0

    return {
        "Accuracy": acc,  # "How often am I correct overall, regardless of class?"
        "Precision": prec,  # "When I flag something, how often is it truly an issue?"
        "Recall / TPR": rec,  # "Out of all actual issues, how many did I correctly flag?"
        "FPR": fpr,  # "How often do I wrongly flag a normal case?"
        "NPV (Negative Predictive Value)": npv,  # "When I say something is normal, how often is that actually true?"
        "Balanced Accuracy": bal_acc,  # "How well do I perform on both classes, accounting for imbalance?"
        "F1-Score": f1,  # "What’s the balance between being thorough and being right when flagging?"
        # "FDR (False Discovery Rate)": fdr,  # "When I raise an alert, how often am I wrong?" FDR = 1 − Precision
        # "FNR (False Negative Rate)": fnr,  # "How often do I miss a real issue?" FNR = 1 − Recall
        # "Specificity / TNR": tnr,  # "Out of all normal cases, how many did I correctly leave alone?" TNR = 1 − FPR
    }



def plot_mcusum_static_historgram(mcusum_statistics, threshold=95, h=0):
    threshold_value = np.percentile(mcusum_statistics, threshold)
    plt.figure(figsize=(12, 6))
    # Main histogram
    sns.histplot(
        mcusum_statistics,
        bins=50,
        kde=True,
        stat="density",
        color="blue",
        edgecolor="black",
        alpha=0.6,
    )
    # Vertical lines
    plt.axvline(h, color="red", linestyle="--", label="Control Limit (h)")
    plt.axvline(threshold_value,
                color="orange",
                linestyle="--",
                label="95th Percentile")
    # Shade area above 95th percentile
    density = (sns.kdeplot(mcusum_statistics,
                           bw_adjust=1).get_lines()[0].get_data())
    plt.fill_between(
        density[0],
        density[1],
        where=density[0] > threshold_value,
        color="orange",
        alpha=0.3,
        label="Top 5%",
    )
    plt.title("MCUSUM Statistics Distribution with Top 5% Highlighted")
    plt.xlabel("MCUSUM Value")
    plt.ylabel("Density")
    plt.legend()
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()


def plot_mcusum_score_as_time_series(mcusum_statistics_in_control,
                                     mcusum_statistics_out_of_control=None,
                                     h=50):
    # Plottting the MCUSUM statiscs as a time series
    plt.figure(figsize=(14, 6))
    plt.plot(
        mcusum_statistics_in_control,
        label="MCUSUM Statistics (INC)",
        color="blue",
    )
    if mcusum_statistics_out_of_control is not None:
        plt.plot(
            mcusum_statistics_out_of_control,
            label="MCUSUM Statistics (OOC)",
            color="green",
            alpha=0.5,
        )
    plt.axhline(h, color="red", linestyle="--", label="Control Limit (h)")
    plt.title("MCUSUM Statistics Over Time")
    plt.xlabel("Sample Index")
    plt.ylabel("MCUSUM Value")
    plt.legend()
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()


In [ ]:
# MCUSUM Configure and play with hyperparameters

# Estimate parameters from in-control data
mu_0, sigma = estimate_incontrol_parameters(X_INCONTROL_TRAIN_REDUCED)

delta = np.zeros(X_INCONTROL_TRAIN_REDUCED.shape[1])  # Initialize delta vector
delta = np.ones(52) * 0.1
k = compute_reference_value_k(delta, sigma)
h = estimate_h(X_INCONTROL_TRAIN_REDUCED, k, 99)  # control limit

mcusum_statistics = compute_mcusum_scores(X_INCONTROL_TEST_REDUCED, mu_0, sigma, k)
mcusum_statistics_ofc = compute_mcusum_scores(X_OUT_OF_CONTROL_TEST_REDUCED, mu_0,
                                              sigma, k)

plot_mcusum_static_historgram(mcusum_statistics=mcusum_statistics,
                              threshold=95,
                              h=h)
plot_mcusum_score_as_time_series(
    mcusum_statistics_in_control=mcusum_statistics,
    mcusum_statistics_out_of_control=mcusum_statistics_ofc,
    h=h,
)

print("Computed k:", k)
print("MCUSUM statistics shape:", mcusum_statistics.shape)
print("First 10 MCUSUM values:", mcusum_statistics[:10])
print("Maximum MCUSUM value:", np.max(mcusum_statistics))

# print 95 percentile of the MCUSUM statistics
mcusum_95_percentile = np.percentile(mcusum_statistics, 95)
print(f"95th Percentile of MCUSUM Statistics: {mcusum_95_percentile:.2f}")

# Flagging out-of-control points
mcusum_flags = mcusum_statistics > h
print(mcusum_flags)
# Output summary
print(f"MCUSUM Control Limit: {h:.2f}")
print(
    f"Out-of-Control Points (MCUSUM): {np.sum(mcusum_flags)} / {len(mcusum_flags)}"
)
print(
    f"Percentage of Out-of-Control Points: {np.mean(mcusum_flags) * 100:.2f}%")
print(
    "No out-of-control points expected, so the percentage should be close to 0%."
)


**MCUSUM could not seprate normal data from fault type 3**

In [ ]:
# MCUSUM Run on test data

mcusum_statistics = compute_mcusum_scores(X_TEST_REDUCED, mu_0, sigma, k)

print("MCUSUM statistics shape:", mcusum_statistics.shape)
print("First 10 MCUSUM values:", mcusum_statistics[:10])

# Flagging out-of-control points
mcusum_flags = mcusum_statistics > h
print(mcusum_flags)
# Output summary
print(f"MCUSUM Control Limit: {h:.2f}")
print(
    f"Out-of-Control Points (MCUSUM): {np.sum(mcusum_flags)} / {len(mcusum_flags)}"
)
print(
    tabulate(
        [[
            np.sum(mcusum_flags),
            len(mcusum_flags),
            np.mean(mcusum_flags) * 100,
        ]],
        headers=["Out-of-Control Points", "Total Points", "Percentage (%)"],
        tablefmt="grid",
    ))

# True labels (0 = normal, 1 = anomaly)
y_true = Y_TEST_ANOMALY_REDUCED_DF.astype(int)

# Predicted labels from MCUSUM
y_pred = mcusum_flags.astype(int)

print("mcusum_flags.shape:", mcusum_flags.shape)
print("y_test_anomaly.shape:", Y_TEST_ANOMALY_REDUCED_DF.shape)

# Compute detection metrics for MCUSUM
print("mcusum_flags.shape:", mcusum_flags.shape)
print("y_test_anomaly.shape:", Y_TEST_ANOMALY_REDUCED_DF.shape)


def compute_detection_metrics(predicted, true_labels: NDArray[np.int64]):
    y_pred = predicted.astype(int)
    y_true = true_labels.astype(int)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc: float = (tp + tn) / (tp + tn + fp + fn)
    prec: float = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    rec: float = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    tnr: float = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    fpr: float = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    fnr: float = fn / (fn + tp) if (fn + tp) > 0 else 0.0
    npv: float = tn / (tn + fn) if (tn + fn) > 0 else 0.0
    fdr: float = fp / (fp + tp) if (fp + tp) > 0 else 0.0
    bal_acc: float = 0.5 * (rec + tnr)
    f1: float = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0.0
    return pd.DataFrame([{
        "Accuracy": acc,  # "How often am I correct overall, regardless of class?"
        "Precision": prec,  # "When I flag something, how often is it truly an issue?"
        "Recall / TPR": rec,  # "Out of all actual issues, how many did I correctly flag?"
        "F1-Score": f1,  # "What’s the balance between being thorough and being right when flagging?"
        "FPR": fpr,  # "How often do I wrongly flag a normal case?"
        "NPV (Negative Predictive Value)": npv,  # "When I say something is normal, how often is that actually true?"
        "Balanced Accuracy": bal_acc,  # "How well do I perform on both classes, accounting for imbalance?"
        # "FDR (False Discovery Rate)": fdr,  # "When I raise an alert, how often am I wrong?" FDR = 1 − Precision
        # "FNR (False Negative Rate)": fnr,  # "How often do I miss a real issue?" FNR = 1 − Recall
        # "Specificity / TNR": tnr,  # "Out of all normal cases, how many did I correctly leave alone?" TNR = 1 − FPR
    }])




mcusum_metrics = compute_detection_metrics(mcusum_flags.astype(int),
                                           Y_TEST_ANOMALY_REDUCED_DF)

# Convert to one-row DataFrame
# mcusum_df = pd.DataFrame([mcusum_metrics])

# Add to the results dict
classification_results_per_model["MCUSUM"] = mcusum_metrics
anomaly_results_per_model["MCUSUM"] = mcusum_metrics


In [ ]:
# Print Metrics Tables

for model_name, df in classification_results_per_model.items():
    # Add an "Average" row at the end of the DataFrame
    average_row = df.mean(axis=0).to_frame().T
    average_row.index = ["Average"]
    df = pd.concat([df, average_row])
    save_dataframe(df=df, name=model_name,suffix="metrics")
    print(f"Classification Metrics for {model_name}:")
    print(tabulate(df, headers="keys", tablefmt="grid"))
    print("\n")

## DNN

In [ ]:
# DNN Model
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping

# Define input layer
inputs = Input(shape=(X_TRAIN.shape[1], ))

# Define hidden layer with 16 nodes and ReLU activation function
hidden_layer = Dense(100, activation="selu")(inputs)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
# Define output layer with softmax activation function for multi-class classification
outputs = Dense(Y_ENC_ANOMALY_TEST_REDUCED.shape[1],
                activation="softmax")(hidden_layer)

# Define the model
model_anomaly = Model(inputs=inputs, outputs=outputs)

# Compile the model with binary cross-entropy loss function and Adam optimizer
model_anomaly.compile(loss="categorical_crossentropy",
                      optimizer="adam",
                      metrics=["accuracy"])

# Print the summary of the model
model_anomaly.summary()

# Define early stopping callback to monitor validation loss and stop if it doesn't improve for 5 epochs
early_stop = EarlyStopping(monitor="val_loss", patience=5)

# Train the model with 20 epochs and batch size of 32, using the early stopping callback
history = model_anomaly.fit(
    X_TRAIN,
    Y_ENC_ANOMALY_TRAIN_REDUCED,
    epochs=200,
    batch_size=256,
    validation_data=(X_TEST_REDUCED, Y_ENC_ANOMALY_TEST_REDUCED),
    callbacks=[early_stop],
)

# Plot the training history for loss and accuracy
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Value")
plt.legend()
plt.show()

In [ ]:
y_pred_anomaly_nn = encoder_2.inverse_transform(
    model_anomaly.predict(X_TEST_REDUCED, verbose=0))
y_true_anomaly_nn = encoder_2.inverse_transform(
    Y_ENC_ANOMALY_TEST_REDUCED)

dnn_anomaly_metrics = compute_detection_metrics(y_pred_anomaly_nn,
                                                y_true_anomaly_nn)

# Convert to one-row DataFrame
# mcusum_df = pd.DataFrame([dnn_anomaly_metrics])

# Add to the results dict
anomaly_results_per_model["DNN"] = dnn_anomaly_metrics

## ML

In [ ]:
# Fit models to the data
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

# Create an instance of each algorithm
# logreg = LogisticRegression(max_iter=10000)
# knn = KNeighborsClassifier()
# dt = DecisionTreeClassifier()
# nb = GaussianNB()
# svm = SVC()

rf = RandomForestClassifier()
xg = xgb.XGBClassifier()

# Train the algorithms on the data
# logreg.fit(x_train, Y_TRAIN)
# knn.fit(x_train, Y_TRAIN)
# dt.fit(x_train, Y_TRAIN)
# nb.fit(x_train, Y_TRAIN)
# svm.fit(x_train, Y_TRAIN)

rf.fit(X_TRAIN, Y_TRAIN_ANOMALY_REDUCED_DF.to_numpy())
xg.fit(X_TRAIN, Y_TRAIN_ANOMALY_REDUCED_DF.to_numpy())

# Use the trained models to make predictions on new data
# y_pred_logreg =logreg.predict(x_test)
# y_pred_dt = dt.predict(x_test)
# y_pred_nb = nb.predict(x_test)
# y_pred_knn = knn.predict(x_test)
# y_pred_svm = svm.predict(x_test)

y_pred_anomaly_rf = rf.predict(X_TEST_REDUCED)
y_pred_anomaly_xg = xg.predict(X_TEST_REDUCED)


In [ ]:
rf_anomaly_metrics = compute_detection_metrics(y_pred_anomaly_rf,
                                               Y_TEST_ANOMALY_REDUCED_DF)
xg_anomaly_metrics = compute_detection_metrics(y_pred_anomaly_xg,
                                               Y_TEST_ANOMALY_REDUCED_DF)

#results_per_model["RF_ANOMALY"] = rf_anomaly_metrics
#results_per_model["XG_ANOMALY"] = xg_anomaly_metrics

anomaly_results_per_model["RF_ANOMALY"] = rf_anomaly_metrics
anomaly_results_per_model["XG_ANOMALY"] = xg_anomaly_metrics

## PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from typing import Tuple


def train_pca_model(
        X_train: NDArray[np.float64],
        n_components=0.9,
        contamination: float = 0.05) -> Tuple[StandardScaler, PCA, float]:
    """
    Trains PCA and scaler on normal training data and computes anomaly threshold.

    Args:
        X_train (NDArray[np.float64]): Training data (normal only or mostly normal).
        n_components (int): PCA components.
        contamination (float): Assumed anomaly rate to define threshold.

    Returns:
        Tuple[StandardScaler, PCA, float]: (fitted scaler, fitted PCA model, anomaly threshold)
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_train)

    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_scaled)
    X_reconstructed = pca.inverse_transform(X_pca)

    errors = np.sum((X_scaled - X_reconstructed)**2, axis=1)
    threshold_index = int((1 - contamination) * len(errors))
    threshold = np.sort(errors)[threshold_index]

    return scaler, pca, threshold


def detect_anomalies_pca(
    X_test: NDArray[np.float64],
    scaler: StandardScaler,
    pca: PCA,
    threshold: float,
) -> Tuple[NDArray[np.float64], NDArray[np.int64]]:
    """
    Applies trained PCA model to test data and flags anomalies.

    Args:
        X_test (NDArray[np.float64]): Test data.
        scaler (StandardScaler): Fitted scaler from training phase.
        pca (PCA): Fitted PCA model.
        threshold (float): Reconstruction error threshold for anomaly.

    Returns:
        Tuple[NDArray[np.float64], NDArray[np.int64]]:
            - reconstruction_errors
            - binary anomaly flags (1 = anomaly)
    """
    X_scaled = scaler.transform(X_test)
    X_pca = pca.transform(X_scaled)
    X_reconstructed = pca.inverse_transform(X_pca)

    reconstruction_errors = np.sum((X_scaled - X_reconstructed)**2, axis=1)
    anomaly_flags = (reconstruction_errors > threshold).astype(int)

    return reconstruction_errors, anomaly_flags


In [ ]:
# Phase 1: Fit
scaler, pca_model, threshold = train_pca_model(X_TRAIN,
                                               n_components=0.9,
                                               contamination=0.05)

# Phase 2: Predict
errors, y_pred = detect_anomalies_pca(X_TEST_REDUCED, scaler, pca_model, threshold)

# Evaluate
pca_metrics_df = compute_detection_metrics(y_pred, Y_TEST_ANOMALY_REDUCED_DF)

#results_per_model["PCA"] = pca_metrics_df
anomaly_results_per_model["PCA"] = pca_metrics_df


## LGB

In [ ]:
import lightgbm as lgb


def train_lgb_model(X_train: NDArray[np.float64],
                    y_train: NDArray[np.int64],
                    seed: int = 42) -> lgb.Booster:
    """
    Trains a LightGBM model and returns the trained booster.

    Args:
        X_train (NDArray[np.float64]): Training features.
        y_train (NDArray[np.int64]): Training labels (0 = normal, 1 = anomaly).
        seed (int): Random seed.

    Returns:
        lgb.Booster: Trained LightGBM model.
    """
    train_set = lgb.Dataset(X_train, label=y_train)

    params = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "learning_rate": 0.1,
        "num_leaves": 31,
        "seed": seed,
    }

    model: lgb.Booster = lgb.train(params, train_set, num_boost_round=100)

    return model


In [ ]:
model_lgb = train_lgb_model(X_TRAIN, Y_TRAIN_ANOMALY_REDUCED_DF)

y_proba: NDArray[np.float64] = model_lgb.predict(X_TEST_REDUCED)
y_pred: NDArray[np.int64] = (y_proba >= 0.5).astype(int)

lgb_metrics = compute_detection_metrics(y_pred, Y_TEST_ANOMALY_REDUCED_DF)
#results_per_model["LGB"] = lgb_metrics
anomaly_results_per_model["LGB"] = lgb_metrics
print(lgb_metrics)

## Autoencoder

In [ ]:
from typing import Optional, List
from tensorflow import keras
from tensorflow.keras import layers


def build_autoencoder_dynamic(
        input_dim: int,
        latent_dim: Optional[int] = None,
        hidden_dims: Optional[List[int]] = None) -> keras.Model:
    """
    Builds an autoencoder with configurable number of hidden layers.

    Args:
        input_dim (int): Number of input features.
        latent_dim (Optional[int]): Size of bottleneck. If None, use input_dim // 20, clamped to [4, 64].
        hidden_dims (Optional[List[int]]): List of encoder layer sizes before bottleneck.
                                           Decoder will mirror them. If None, a default pattern is used.

    Returns:
        keras.Model: Compiled autoencoder model.
    """
    # Determine latent size if not provided
    if latent_dim is None:
        latent_dim = max(4, min(64, input_dim // 20))

    # Set default encoder sizes if not provided
    if hidden_dims is None:
        # Example: for input_dim=100, get [64, 32]
        hidden_dims = [max(32, input_dim // 2), max(16, input_dim // 4)]

    # Define input layer
    input_layer = keras.Input(shape=(input_dim, ))
    x = input_layer

    # Encoder: stack each Dense layer with ReLU
    for size in hidden_dims:
        x = layers.Dense(size, activation='relu')(x)

    # Bottleneck layer
    encoded = layers.Dense(latent_dim, activation='relu')(x)

    # Decoder: mirror of encoder
    for size in reversed(hidden_dims):
        encoded = layers.Dense(size, activation='relu')(encoded)

    # Final output layer: linear activation to reconstruct inputs
    output_layer = layers.Dense(input_dim, activation='linear')(encoded)

    # Construct and compile model
    model = keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='mse')

    return model


def train_autoencoder_tf(X_train: np.ndarray,
                         model: keras.Model,
                         n_epochs: int = 100,
                         batch_size: int = 64,
                         patience: int = 10,
                         verbose: bool = True) -> keras.Model:
    """
    Trains the Keras autoencoder using early stopping and prints debug output.

    Args:
        X_train (np.ndarray): Input training data, shape (n_samples, n_features).
        model (keras.Model): Keras autoencoder model instance to train.
        n_epochs (int): Maximum number of training epochs.
        batch_size (int): Number of samples per batch during training.
        patience (int): Number of epochs with no improvement before stopping early.
        verbose (bool): If True, print debug and training status messages.

    Returns:
        keras.Model: Trained model with best weights.
    """
    # Ensure training data is in float32 (required by TensorFlow)
    if X_train.dtype != np.float32:
        X_train = X_train.astype(np.float32)
    if verbose:
        print(f"[DEBUG] Input shape: {X_train.shape}, dtype: {X_train.dtype}")

    # Create early stopping callback to halt training when loss plateaus
    early_stop = keras.callbacks.EarlyStopping(
        monitor="loss",  # monitor training loss
        patience=patience,  # stop if no improvement for N epochs
        min_delta=1e-6,  # only improvements greater than this count
        restore_best_weights=True,  # restore weights from lowest loss
        verbose=1 if verbose else 0)

    # Fit model on training data using input = output (unsupervised)
    model.fit(
        X_train,  # input data
        X_train,  # target is same as input (autoencoder)
        epochs=n_epochs,  # maximum training epochs
        batch_size=batch_size,  # batch size
        shuffle=True,  # shuffle input data each epoch
        callbacks=[early_stop],  # attach early stopping
        verbose=1 if verbose else 0  # print training logs if requested
    )

    return model


def compute_reconstruction_error(model: keras.Model,
                                 X: np.ndarray) -> np.ndarray:
    """
    Computes squared reconstruction error for each sample.

    Args:
        model (keras.Model): Trained autoencoder model.
        X (np.ndarray): Input data (e.g., test set), shape (n_samples, n_features).

    Returns:
        np.ndarray: Reconstruction error (per sample), shape (n_samples,).
    """
    # Ensure float32 input (required by TensorFlow)
    if X.dtype != np.float32:
        X = X.astype(np.float32)

    # Predict reconstructed input using the trained model
    X_reconstructed: np.ndarray = model.predict(X, verbose=0)

    # Compute element-wise squared error per sample (L2 norm)
    squared_error: np.ndarray = np.sum((X - X_reconstructed)**2, axis=1)

    return squared_error


def detect_anomalies_from_error(errors: np.ndarray,
                                contamination: float = 0.05) -> np.ndarray:
    """
    Flags anomalies based on sorted reconstruction error and contamination rate.

    Args:
        errors (np.ndarray): Reconstruction error per sample.
        contamination (float): Expected proportion of anomalies.

    Returns:
        np.ndarray: Binary array of flags (1 = anomaly, 0 = normal).
    """
    # Compute the threshold (95th percentile if contamination = 0.05)
    threshold_idx = int((1 - contamination) * len(errors))
    threshold = np.sort(errors)[threshold_idx]

    # Flag values above threshold as anomalies
    return (errors > threshold).astype(int)


def compute_latent_dim(input_dim: int) -> int:
    """
    Computes a reasonable latent dimension based on input complexity.

    Args:
        input_dim (int): Number of features in the input.

    Returns:
        int: Latent dimension for autoencoder.
    """
    # Rule: at least 4, at most 64, about 1/20th of input
    return max(4, min(64, input_dim // 20))


In [ ]:
input_dim: int = X_TRAIN.shape[1]  # dynamically detect feature count
latent_dim = compute_latent_dim(input_dim)
autoencoder_model = build_autoencoder_dynamic(input_dim=X_TRAIN.shape[1])
autoencoder_model = train_autoencoder_tf(X_TRAIN, autoencoder_model)

# After training

# 1. Predict reconstruction error on x_test
errors_test = compute_reconstruction_error(autoencoder_model, X_TEST_REDUCED)

# 2. Convert reconstruction errors to binary anomaly predictions
y_pred: np.ndarray = detect_anomalies_from_error(errors_test,
                                                 contamination=0.05)

# 3. Use your true binary test labels (0 = normal, 1 = anomaly)
#     Example: y_test_anomaly = np.array([0, 0, 1, 0, ...])
#     It must match x_test.shape[0]

# 4. Evaluate metrics using your benchmark function
autoencoder_metrics = compute_detection_metrics(y_pred, Y_TEST_ANOMALY_REDUCED_DF)

# 5. Optionally print or tabulate results
#results_per_model["Autoencoder"] = autoencoder_metrics
anomaly_results_per_model["Autoencoder"] = autoencoder_metrics
print(autoencoder_metrics)


## Evaluating


---

**Detailed Metric Breakdown**

| **Metric**                          | **Formula**                                                                             | **Interpretation in Fault Detection**                                                                                                                                              |
| ----------------------------------- | --------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Accuracy**                        | $\frac{TP + TN}{TP + TN + FP + FN}$                                                     | Overall correctness. Can be misleading when faults are rare. High accuracy might simply mean the model always predicts "normal."                                                   |
| **Precision**                       | $\frac{TP}{TP + FP}$                                                                    | Of all alarms raised, how many were actual faults? **Low precision means many false alarms**, which overload operators and reduce trust in the system.                             |
| **Recall / Sensitivity / TPR**      | $\frac{TP}{TP + FN}$                                                                    | Proportion of actual faults correctly detected. **High recall is critical in safety-sensitive environments** — a low recall means faults go undetected, risking damage or hazards. |
| **Specificity / TNR**               | $\frac{TN}{TN + FP}$                                                                    | Proportion of normal conditions correctly identified. High specificity = fewer false alarms, preserving operator trust and avoiding unnecessary shutdowns.                         |
| **FPR (False Positive Rate)**       | $\frac{FP}{FP + TN}$                                                                    | Rate at which the system wrongly flags normal data as faulty. High FPR leads to wasted investigations and “alarm fatigue.”                                                         |
| **FNR (False Negative Rate)**       | $\frac{FN}{FN + TP}$                                                                    | Rate at which faults are missed. **The most dangerous metric** in safety-critical monitoring. A high FNR means faults are silently ignored.                                        |
| **NPV (Negative Predictive Value)** | $\frac{TN}{TN + FN}$                                                                    | When the system says “everything is fine,” how often is it actually correct? Important for trusting normal-state decisions, especially when faults are rare.                       |
| **FDR (False Discovery Rate)**      | $\frac{FP}{FP + TP}$                                                                    | Among all detected anomalies, how many were false? High FDR undermines the system’s credibility. Operators may start ignoring the model’s outputs.                                 |
| **Balanced Accuracy**               | $\frac{1}{2} (\text{TPR} + \text{TNR})$                                                 | Averages recall for both classes (normal and fault). Gives fair performance measurement on **imbalanced data** (e.g., few faults).                                                 |
| **F1 Score**                        | $\frac{2 \cdot \text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$ | Harmonizes Precision and Recall. Penalizes when either false alarms or missed faults are high. **Best for evaluating fault classifiers where both errors are costly.**             |

---

🔍 Use-Case Driven Summary

| **If you want to...**                          | **Use this metric...**    | **Because...**                                        |
| ---------------------------------------------- | ------------------------- | ----------------------------------------------------- |
| Detect all faults without missing any          | **Recall (TPR)**          | Missing a fault could cause damage or loss.           |
| Avoid false alarms                             | **Precision, FPR, FDR**   | Too many false positives cause alarm fatigue.         |
| Know how trustworthy “normal” output is        | **NPV**                   | Operators rely on normal predictions to avoid action. |
| Compare performance across imbalanced datasets | **Balanced Accuracy, F1** | Raw accuracy is biased when faults are rare.          |

---

If both **Recall = 1.0** and **Precision = 1.0**, this means:

* **Every actual fault was correctly detected (no false negatives)**
* **Every predicted fault was correct (no false positives)**

### Table Metrics

In [ ]:
# Print Metrics Tables


def convert_result_dict_to_df(results_dict):
    results_df = pd.concat(
        [df.assign(Model=model) for model, df in results_dict.items()],
        ignore_index=True)
    # Move 'Model' column to the first position after index
    cols = ['Model'] + [col for col in results_df.columns if col != 'Model']
    results_df = results_df[cols]
    return results_df

    cols = ['Model'] + [col for col in conctated_anomaly_results_df.columns if col != 'Model']
conctated_anomaly_results_df = convert_result_dict_to_df(anomaly_results_per_model)
save_dataframe(df=conctated_anomaly_results_df,name="Anomaly detection metrics")
conctated_anomaly_results_df.head()


### Plot Metric

In [ ]:
# plot_model_anomaly_detection_comparison

import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict


def plot_model_anomaly_detection_comparison(
    metrics_dict: Dict[str, pd.DataFrame], ) -> None:
    df_combined = pd.concat(
        [df.assign(Model=model) for model, df in metrics_dict.items()],
        ignore_index=True,
    )

    df_melted = df_combined.melt(id_vars="Model",
                                 var_name="Metric",
                                 value_name="Value")

    plt.figure(figsize=(14, 6))
    ax = plt.subplot()

    model_list = df_melted["Model"].unique().tolist()
    metric_list = df_melted["Metric"].unique().tolist()
    n_models = len(model_list)
    bar_width = 0.2
    group_spacing = 0.4  # Gap between metric groups

    # X-axis base positions for each metric group
    base_positions = [
        i * (n_models * bar_width + group_spacing)
        for i in range(len(metric_list))
    ]

    # Plot each model
    for model_idx, model in enumerate(model_list):
        subset = df_melted[df_melted["Model"] == model]

        # Compute shifted bar positions for this model within each group
        bar_positions = [pos + bar_width * model_idx for pos in base_positions]

        ax.bar(bar_positions, subset["Value"], width=bar_width, label=model)

    # Set proper x-ticks centered under each metric group
    tick_positions = [
        pos + (bar_width * n_models / 2) - (bar_width / 2)
        for pos in base_positions
    ]
    ax.set_xticks(tick_positions)
    ax.set_xticklabels(metric_list, rotation=45, ha="right")

    ax.set_ylabel("Score")
    ax.set_title("Model Comparison Across Metrics")
    ax.legend()
    ax.yaxis.grid(True, linestyle="--", alpha=0.7)
    plt.tight_layout()
    save_plot(plot_name="Anomaly detection metrics", plot_path="anomaly")
    plt.show()


plot_model_anomaly_detection_comparison(anomaly_results_per_model)

# Faults merging

## Neural Network

In [ ]:
# DNN Model
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping

# Define input layer
inputs = Input(shape=(X_TRAIN.shape[1], ))

# Define hidden layer with 16 nodes and ReLU activation function
hidden_layer = Dense(100, activation="selu")(inputs)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
hidden_layer = Dense(100, activation="selu")(hidden_layer)
# Define output layer with softmax activation function for multi-class classification
outputs = Dense(Y_ENC_MERGED_TRAIN_REDUCED.shape[1],activation="softmax")(hidden_layer)

# Define the model
model_fm = Model(inputs=inputs, outputs=outputs)

# Compile the model with binary cross-entropy loss function and Adam optimizer
model_fm.compile(loss="categorical_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])

# Print the summary of the model
model_fm.summary()

# Define early stopping callback to monitor validation loss and stop if it doesn't improve for 5 epochs
early_stop = EarlyStopping(monitor="val_loss", patience=5)

# Train the model with 20 epochs and batch size of 32, using the early stopping callback
history = model_fm.fit(
    X_TRAIN,
    Y_ENC_MERGED_TRAIN_REDUCED,
    epochs=200,
    batch_size=256,
    validation_data=(X_TEST_REDUCED, Y_ENC_MERGED_TEST_REDUCED),
    callbacks=[early_stop],
)

# Plot the training history for loss and accuracy
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.plot(history.history["accuracy"], label="Training Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Value")
plt.legend()
plt.show()

In [ ]:
nn_results_fm = model_fm.predict(X_TEST_REDUCED, verbose=0)
y_pred_nn_fm = encoder_3.inverse_transform(nn_results_fm)
y_true_nn_fm= encoder_3.inverse_transform(Y_ENC_MERGED_TEST_REDUCED)
y_score_nn_fm = model_fm.predict(X_TEST_REDUCED, verbose=0)

## Evaluating

In [ ]:
evaluate_model("Neural Net FM", y_true_nn_fmy_pred_nn.ravel(), y_pred_nn_fmy_pred_nn.ravel())

### Plots per fault per Metric

In [ ]:
plot_all_fault_scores_per_metric_all_models(classification_scores_per_fault_tables_dict)

### Table Metric

In [ ]:
#results_per_model["Neural Net FM"] = compute_classification_metrics_per_class(y_true_nn_fm, y_pred_nn_fm)

# loop through each model and print the classification metrics
for model_name, df in classification_results_per_model.items():
    # Add an "Average" row at the end of the DataFrame
    average_row = df.mean(axis=0).to_frame().T
    average_row.index = ["Average"]
    df = pd.concat([df, average_row])
    save_dataframe(df=df, name=model_name,suffix="metrics")
    print(f"Classification Metrics for {model_name}:")
    print(tabulate(df, headers="keys", tablefmt="grid"))
    print("\n")

### Confusion matrices

In [ ]:
plot_all_confusion_matrices({
    "Neural Net": (y_true_nn, y_pred_nn),
    "Neural Net FM": (y_true_nn_fm, y_pred_nn_fm),
})